In [ ]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy
import sklearn
import itertools 
from itertools import cycle 
import os.path as op
import timeit 
import json
import math

In [ ]:
import multiprocessing as m_proc
m_proc.cpu_count()

In [ ]:
# Import MDAnalysis
import MDAnalysis as mda
import statsmodels as stats

In [ ]:
from plgapeg_MD_analysis import (pers_length, get_rg_pers_poly, bavg_pers_cnt, pos_bead_autocorr_RA,
                                 rouse_relax, zimm_relax_fit, zimm_relax_func, rh_block_avg, mean_sq_e2e, hydro_rad_poly,
                                obs_autocorr_RA)

In [ ]:
from polymer_MD_analysis import (pers_length, get_rg_pers_poly, bavg_pers_cnt, pos_bead_autocorr_RA,
                                 rouse_relax, zimm_relax_fit, zimm_relax_func, rh_block_avg, mean_sq_e2e, hydro_rad_poly,
                                obs_autocorr_RA, getPairStats, pair_regression, simple_regression_conf)

## PLGA-PEG/DMSO system analysis

### N = 6 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n6_plgapeg_dmso = mda.Universe("n6plga_peg_dmso/n6plgapeg_DMSO.pdb", "n6plga_peg_dmso/NoDMSO_n6plgapeg.xtc")

In [ ]:
n6_plgapeg_dmso.trajectory

In [ ]:
len(n6_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n6dmso = n6_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n6dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n6plgapeg_dmso = pers_length(plgapeg_n6dmso,6)
crv_n6plgapeg_dmso

### Orientation Order parameter vs. time 

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
OOp_n6plgapeg_dmso = orientation_order_param(plgapeg_n6dmso, n6_plgapeg_dmso, n_mon, start, end)

In [ ]:
OOp_n6plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 6 PLGA-PEG in DMSO

In [ ]:
OOp_n6plgapeg_dmso[82]

In [ ]:
OOp_n6plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n6plgapeg_dmso)

In [ ]:
np.std(OOp_n6plgapeg_dmso)

### Mean squared end to end distance 

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
eVec_n6plgapeg_dmso, e2edis_n6plgapeg_dmso = mean_sq_e2e(plgapeg_n6dmso, n6_plgapeg_dmso, n_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Ree Autocorrelation function

In [ ]:
e2edis_n6plgapeg_dmso[1]

In [ ]:
t_corr = 5000
window_shift = 15

In [ ]:
ACn6plgapegdmso_Ree, eqvar_n6plgapegdmso = obs_autocorr_RA(e2edis_n6plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegdmso_Ree

In [ ]:
eqvar_n6plgapegdmso

In [ ]:
ACn6plgapegdmso_sqRee, evsqRee_n6plgapegdmso = obs_autocorr_RA(e2edis_n6plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n6plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,70)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
plgapegn6dmso_rh = hydro_rad_poly(plgapeg_n6dmso, n6_plgapeg_dmso, n_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
t_corr = 5000
window_shift = 9

In [ ]:
ACn6plgapegdmsoRh, eqvarRh_n6plgapegdmso = obs_autocorr_RA(plgapegn6dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegdmsoRh

In [ ]:
eqvarRh_n6plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='teal')
#plt.plot(A2n6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='r')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,70)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n_mon = 6 
start = 0
end = 36001
t_corr = 1000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn6dmso, tcSUM_plgapegn6dmso = pos_bead_autocorr_RA(plgapeg_n6dmso, n6_plgapeg_dmso, n_mon, t_corr, window_shift
                                                                , start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn6dmso

In [ ]:
tcSUM_plgapegn6dmso.shape

### Fitting autocorrelation data 

In [ ]:
tcRA_plgapegn6dmso = np.load('n6ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn8dmso = np.load('n8ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn10dmso = np.load('n10ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn16dmso = np.load('n16ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn20dmso = np.load('n20ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn20dmsoT2 = np.load('n20ppdmsoT2_CtRA.npy', allow_pickle=True)
tcRA_plgapegn30dmso = np.load('n30ppdmso_CtRA.npy', allow_pickle=True)
tcRA_plgapegn30dmsoT2 = np.load('n30ppdmsoT2_CtRA.npy', allow_pickle=True)

In [ ]:
s_n6 =[2 for i in range(tcRA_plgapegn6dmso[1].shape[0])]
s_n8 =[2 for i in range(tcRA_plgapegn8dmso[1].shape[0])]
s_n10 =[2 for i in range(tcRA_plgapegn10dmso[1].shape[0])]
s_n16 =[2 for i in range(tcRA_plgapegn16dmso[1].shape[0])]
s_n20 =[2 for i in range(tcRA_plgapegn20dmso[1].shape[0])]
sT2_n20 =[2 for i in range(tcRA_plgapegn20dmsoT2[1].shape[0])]
s_n30 =[2 for i in range(tcRA_plgapegn30dmso[1].shape[0])]
sT2_n30 =[2 for i in range(tcRA_plgapegn30dmsoT2[1].shape[0])]

In [ ]:
len(s_n30)

In [ ]:
tcRA_plgapegn20dmso[0].shape

In [ ]:
tcRA_plgapegn20dmsoT2[0].shape

In [ ]:
np.intersect1d(tcRA_plgapegn20dmso[0], tcRA_plgapegn20dmsoT2[0])

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(tcRA_plgapegn6dmso[1]/100, tcRA_plgapegn6dmso[0], color='teal', s=s_n6, label=r'$N_{PLGA-PEG}$ = 6')
plt.scatter(tcRA_plgapegn8dmso[1]/100, tcRA_plgapegn8dmso[0], color='dodgerblue', s=s_n8, label=r'$N_{PLGA-PEG}$ = 8')
plt.scatter(tcRA_plgapegn10dmso[1]/100, tcRA_plgapegn10dmso[0], color='darkmagenta', s=s_n10, label=r'$N_{PLGA-PEG}$ = 10')
plt.scatter(tcRA_plgapegn16dmso[1]/100, tcRA_plgapegn16dmso[0], color='maroon', s=s_n16, label=r'$N_{PLGA-PEG}$ = 16')
plt.scatter(tcRA_plgapegn20dmso[1]/100, tcRA_plgapegn20dmso[0], color='orangered', s=s_n20, label=r'$N_{PLGA-PEG}$ = 20')
#plt.scatter(tcRA_plgapegn20dmsoT2[1]/100, tcRA_plgapegn20dmsoT2[0], color='b', s=sT2_n20, label=r'$N_{PLGA-PEG}$ = 20')
plt.scatter(tcRA_plgapegn30dmso[1]/100, tcRA_plgapegn30dmso[0], color='darkolivegreen', s=s_n30, label=r'$N_{PLGA-PEG}$ = 30')
#plt.scatter(tcRA_plgapegn30dmsoT2[1]/100, tcRA_plgapegn30dmsoT2[0], color='green', s=sT2_n30, label=r'$N_{PLGA-PEG}$ = 30')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,360)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
xdata_plgapegn6dmso = tcRA_plgapegn6dmso[1]/100
ydata_plgapegn6dmso = tcRA_plgapegn6dmso[0]

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
ts = 1
hs = 0.1
n_mon = 6
n_b = n_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn6dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 5
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
from lmfit import Minimizer, Parameters, report_fit

In [ ]:
ppfit_n6dmso = Parameters()
#ppfit_n6dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n6dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn6dmso = Minimizer(res_plgapegn6dmso, ppfit_n6dmso, fcn_args=(xdata_plgapegn6dmso, ydata_plgapegn6dmso))
out_ppn6dmso = mini_ppn6dmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(out_ppn6dmso.params)

In [ ]:
out_ppn6dmso.params

In [ ]:
# with h varying, tr = 1.74876099 +/- 0.01504971, h* = 0.13846790 +/- 0.02079379
tdmso_n6plgapeg = []
n_bonds = 5
for i in range(len(xdata_plgapegn6dmso)):
    tdmso_n6plgapeg.append(zimm_relax_fit(xdata_plgapegn6dmso[i], 1.69732007, 0.26,  n_bonds))


In [ ]:
t_c, tz_n6plgapeg, zimm_n6plgapeg, rouse_n6plgapeg = zimm_relax_func(1, 1.69732007, 0.26,  n_bonds)

In [ ]:
x_pos = np.arange(5)
width = 0.25
aa_types = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$"]
fig = plt.figure(figsize=(7,7))
plt.bar(x_pos, tz_n6plgapeg, width, ecolor='black',capsize=5, color='c')
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Relaxation times', fontsize=15)

In [ ]:
x_pos = np.arange(5)
width = 0.25
aa_eigen_ppn6dmso = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA PEG in DMSO N = 6', fontsize=18)
plt.bar(x_pos, rouse_n6plgapeg, width, ecolor='black',capsize=5, color='c', label='Rouse' )
plt.bar(x_pos+width, zimm_n6plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos, labels=aa_eigen_ppn6dmso, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.title(r'Positional bead autocorrelation PLGA PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen6 = np.array([ydata_plgapegn6dmso - np.array(tdmso_n6plgapeg)])
#rpdmso_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.title(r'Relaxation time Fitting Residuals PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,100])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

### Radius of Gyration vs. time N = 6 PLGA-PEG in DMSO

In [ ]:
# Key variables 
n_mon = 6 
start = 0
end = 36001

In [ ]:
n6plgapeg_rgens_dmso, cor_n6plgapeg_dmso, N6plgapeg_cos_dmso, rgdmso_n6plgapeg = get_rg_pers_poly(plgapeg_n6dmso, n6_plgapeg_dmso,
                                                                                                  start,end)

In [ ]:
n6plgapeg_rgens_dmso[0].shape

In [ ]:
cor_n6plgapeg_dmso[3]

In [ ]:
N6plgapeg_cos_dmso

In [ ]:
rgdmso_n6plgapeg

In [ ]:
n6plgapeg_rgens_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
t_corr = 5000
window_shift = 15

In [ ]:
ACn6plgapegdmso_Rg, eqvarRg_n6plgapegdmso = obs_autocorr_RA(n6plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegdmso_Rg

In [ ]:
eqvarRg_n6plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,70)
plt.ylim(-1,1)

#### Get average COM distance, Arc length values

In [ ]:
cb_n6plga = np.zeros(shape=(36001))
cb_n6pplinker = np.zeros(shape=(36001))
cb_n6peg = np.zeros(shape=(36001))
count = 0
for ts in n6_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n6m1_plga = n6_plgapeg_dmso.select_atoms("resid 1")
    n6m2_plga = n6_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker
    n6pl_link = n6_plgapeg_dmso.select_atoms("resid 3")
    n6peg_link = n6_plgapeg_dmso.select_atoms("resid 4")
    # last two PEG residues 
    n6l1_plgapeg = n6_plgapeg_dmso.select_atoms("resid 5")
    n6l2_plgapeg = n6_plgapeg_dmso.select_atoms("resid 6")
    
    cb_n6plga[count] = mda.analysis.distances.distance_array(n6m1_plga.center_of_mass(), n6m2_plga.center_of_mass(), 
                                                   box=n6_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n6pplinker[count] = mda.analysis.distances.distance_array(n6pl_link.center_of_mass(), n6peg_link.center_of_mass(), 
                                                   box=n6_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n6peg[count] = mda.analysis.distances.distance_array(n6l1_plgapeg.center_of_mass(), n6l2_plgapeg.center_of_mass(), 
                                                   box=n6_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn6dmso = np.zeros(shape=(3,2))
COM_ppn6dmso[0,0] = np.mean(cb_n6plga)
COM_ppn6dmso[1,0] = np.mean(cb_n6pplinker)
COM_ppn6dmso[2,0] = np.mean(cb_n6peg)
COM_ppn6dmso[0,1] = np.std(cb_n6plga)
COM_ppn6dmso[1,1] = np.std(cb_n6pplinker)
COM_ppn6dmso[2,1] = np.std(cb_n6peg)
COM_ppn6dmso

#### Correlation values at each arc length from the whole 360 ns trajectory, N = 6 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn6dmso = np.zeros(shape=5)
for i in range(n_mon-1):
    if i < int((n_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn6dmso[i] = 0
        elif i != 0:
            blen_ppn6dmso[i] =  np.sum(blen_ppn6dmso[i-1]) + COM_ppn6dmso[0,0]    
    elif i == int((n_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn6dmso[i] = np.sum(blen_ppn6dmso[i-1]) + COM_ppn6dmso[1,0]
    elif i > int((n_mon/2)-1):
    #    # PEG contour length values
        blen_ppn6dmso[i] = np.sum(blen_ppn6dmso[i-1]) + COM_ppn6dmso[2,0]
        
blen_ppn6dmso

In [ ]:
# Error prop. into natural log std deviation
mk_n6ppdmso = cor_n6plgapeg_dmso[1]/cor_n6plgapeg_dmso[0]
mk_n6ppdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n6_blksplgapeg_dmso , n6plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n6dmso, n6_plgapeg_dmso, blen_ppn6dmso, 3, 0 , 36001)

In [ ]:
n6_blksplgapeg_dmso

In [ ]:
n6plgapeg_lpdmso

In [ ]:
n6plgapeg_lpdmso[2]

In [ ]:
np.mean(n6plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
def line_fit(slope, x):
    return slope*x 

In [ ]:
blen_ppn6dmso

In [ ]:
gg_n6plgapeg_dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn6dmso)

In [ ]:
gg_n6plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 6 PLGA-PEG/DMSO

In [ ]:
blppn6dmso = []
for i in range(n_mon-1):
    if i < int((n_mon/2)-1):
        # PLGA contour length values
        blppn6dmso.append(COM_ppn6dmso[0,0])    
    elif i == int((n_mon/2)-1):
        #PLGA-PEG linker length
        blppn6dmso.append(COM_ppn6dmso[1,0]) 
    elif i > int((n_mon/2)-1):
    #    # PEG contour length values
        blppn6dmso.append(COM_ppn6dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn6dmso = np.sum(blppn6dmso)
lc_ppn6dmso

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso = np.zeros(shape=(3,6))
re2e_rgsq_plgapegdmso[1,0] = lc_ppn6dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn6dmso = np.mean(n6_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn6dmso = np.mean(n6_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,0] = a_ppn6dmso/b_ppn6dmso
re2e_rgsq_plgapegdmso

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso = np.zeros(shape=(3,6))
rhy_rg_plgapegdmso[1,0] = lc_ppn6dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn6dmso = np.mean(n6_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn6dmso = np.mean(n6_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,0] = aa_ppn6dmso/bb_ppn6dmso
rhy_rg_plgapegdmso

In [ ]:
# Kuhn length array at different monomer lengths
kuhnlpL_plgapegdmso = np.zeros(shape=(3,6))
kuhnlpL_plgapegdmso[1] = np.array([6,8,10,16,20,30])
kuhnlpL_plgapegdmso

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,0] = np.mean(n6_blksplgapeg_dmso["Avg persistence length"])/lc_ppn6dmso
kuhnlpL_plgapegdmso

#### Block averaged Radius of gyration and persistence length, N = 6 PLGA-PEG/DMSO

In [ ]:
np.mean(n6_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n6_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n6_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n6_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn6dmso, gg_n6plgapeg_dmso, color='teal')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
plt.legend([r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
#plt.text(0.5, -6.94,r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.61 $\AA$', fontsize=15, color='#1F2E69')

In [ ]:
rgplgapeg_olig_dmso =  pd.DataFrame(data=n6_blksplgapeg_dmso["Avg Radius of gyration"]
                                    , columns=['$R_{g}$ [Angstrom] N = 6 PLGA-PEG DMSO'])
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso = pd.DataFrame(data=n6_blksplgapeg_dmso["Avg persistence length"]
                                  , columns=[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG DMSO"])
pers_plgapegt_dmso

### N = 8 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n8_plgapeg_dmso = mda.Universe("n8plga_peg_dmso/n8plgapeg_DMSO.pdb", "n8plga_peg_dmso/NoDMSO_n8plgapeg.xtc")

In [ ]:
n8_plgapeg_dmso.trajectory

In [ ]:
len(n8_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n8dmso = n8_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n8dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n8plgapeg_dmso = pers_length(plgapeg_n8dmso,8)
crv_n8plgapeg_dmso

### Orientation Order parameter vs. time 

In [ ]:
n8_mon = 8 
start = 0
end = 36001

In [ ]:
OOp_n8plgapeg_dmso = orientation_order_param(plgapeg_n8dmso, n8_plgapeg_dmso, n8_mon, start, end)

In [ ]:
OOp_n8plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_dmso,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 8 PLGA-PEG in DMSO

In [ ]:
OOp_n8plgapeg_dmso[82]

In [ ]:
OOp_n8plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n8plgapeg_dmso)

In [ ]:
np.std(OOp_n8plgapeg_dmso)

### Mean squared end to end distance 

In [ ]:
eVec_n8plgapeg_dmso, e2edis_n8plgapeg_dmso = mean_sq_e2e(plgapeg_n8dmso, n8_plgapeg_dmso, n8_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_dmso[1]/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Ree Autocorrelation function

In [ ]:
e2edis_n8plgapeg_dmso[1]

In [ ]:
ACn8plgapegdmso_Ree, eqvar_n8plgapegdmso = obs_autocorr_RA(e2edis_n8plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegdmso_Ree

In [ ]:
eqvar_n8plgapegdmso

In [ ]:
ACn8plgapegdmso_sqRee, evsqRee_n8plgapegdmso = obs_autocorr_RA(e2edis_n8plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n8plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmso_Ree[1]/100, ACn8plgapegdmso_Ree[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn8dmso_rh = hydro_rad_poly(plgapeg_n8dmso, n8_plgapeg_dmso, n8_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8dmso_rh/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
t_corr = 4000
window_shift = 15

In [ ]:
ACn8plgapegdmsoRh, eqvarRh_n8plgapegdmso = obs_autocorr_RA(plgapegn8dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegdmsoRh

In [ ]:
eqvarRh_n8plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmsoRh[1]/100, ACn8plgapegdmsoRh[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO','N = 8 in DMSO'], frameon=False, fontsize=14)
plt.xlim(0,50)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n8_mon = 8 
start = 0
end = 36001
t_corr = 1000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn8dmso, tcSUM_plgapegn8dmso = pos_bead_autocorr_RA(plgapeg_n8dmso, n8_plgapeg_dmso, n8_mon, t_corr, window_shift, start
                                                               , end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn8dmso

In [ ]:
tcSUM_plgapegn8dmso.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn8dmso = tcRA_plgapegn8dmso[1]/100
ydata_plgapegn8dmso = tcRA_plgapegn8dmso[0]

In [ ]:
ydata_plgapegn8dmso.shape

In [ ]:
xdata_plgapegn8dmso.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
ts = 1
hs = 0.1
n8_mon = 8
n_b = n8_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn8dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 7
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n8dmso = Parameters()
#ppfit_n8dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n8dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn8dmso = Minimizer(res_plgapegn8dmso, ppfit_n8dmso, fcn_args=(xdata_plgapegn8dmso, ydata_plgapegn8dmso))
out_ppn8dmso = mini_ppn8dmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(out_ppn8dmso.params)

In [ ]:
out_ppn8dmso.params

In [ ]:
# with h varying, tr = 2.59052650 +/- 0.01938150, h* = 0.25999998 +/- 0.00998461
tdmso_n8plgapeg = []
n_bonds = 7
for i in range(len(xdata_plgapegn8dmso)):
    tdmso_n8plgapeg.append(zimm_relax_fit(xdata_plgapegn8dmso[i], 2.59078779, 0.26,  n_bonds))


In [ ]:
t_c, tz_n8plgapeg, zimm_n8plgapeg, rouse_n8plgapeg = zimm_relax_func(1, 2.59078779, 0.26,  n_bonds)

In [ ]:
x_pos_n6 = np.arange(5)
x_pos_n8 = np.arange(7)
width = 0.25
aa_types_n8 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO', fontsize=18)
plt.bar(x_pos_n6+width, tz_n6plgapeg, width, ecolor='black',capsize=5, color='teal', label="N = 6")
plt.bar(x_pos_n8, tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.xticks(x_pos_n6+width, labels=aa_types, fontsize=12)
plt.xticks(x_pos_n8, labels=aa_types_n8, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n8 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO N = 8', fontsize=18)
plt.bar(x_pos_n8, rouse_n8plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n8+width, zimm_n8plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n8+width,labels=aa_eigen_n8, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8dmso, tdmso_n8plgapeg, color='dodgerblue')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen8 = np.array([ydata_plgapegn8dmso - np.array(tdmso_n8plgapeg)])
#rpdmso_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, rppdmso_msen8, color='dodgerblue', s=s_n8)
plt.title(r'Relaxation time Fitting Residuals PlGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,100])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 8 PLGA-PEG in DMSO

In [ ]:
# Key variables 
n8_mon = 8 
start = 0
end = 36001

In [ ]:
n8plgapeg_rgens_dmso, cor_n8plgapeg_dmso, N8plgapeg_cos_dmso, rgdmso_n8plgapeg = get_rg_pers_poly(plgapeg_n8dmso, n8_plgapeg_dmso,
                                                                                                  start,end)

In [ ]:
n8plgapeg_rgens_dmso[0].shape

In [ ]:
cor_n8plgapeg_dmso[3]

In [ ]:
N8plgapeg_cos_dmso

In [ ]:
rgdmso_n8plgapeg

In [ ]:
n8plgapeg_rgens_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_dmso[0]/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
t_corr = 4000
window_shift = 15

In [ ]:
ACn8plgapegdmso_Rg, eqvarRg_n8plgapegdmso = obs_autocorr_RA(n8plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegdmso_Rg

In [ ]:
eqvarRg_n8plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmso_Rg[1]/100, ACn8plgapegdmso_Rg[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

#### Get average COM distance, Contour length calc

In [ ]:
cb_n8plga = np.zeros(shape=(36001))
cb_n8pplinker = np.zeros(shape=(36001))
cb_n8peg = np.zeros(shape=(36001))
count = 0
for ts in n8_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n8m1_plga = n8_plgapeg_dmso.select_atoms("resid 1")
    n8m2_plga = n8_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n8pl_link = n8_plgapeg_dmso.select_atoms("resid 4")
    n8peg_link = n8_plgapeg_dmso.select_atoms("resid 5")
    # last two PEG residues 
    n8l1_plgapeg = n8_plgapeg_dmso.select_atoms("resid 7")
    n8l2_plgapeg = n8_plgapeg_dmso.select_atoms("resid 8")
    
    cb_n8plga[count] = mda.analysis.distances.distance_array(n8m1_plga.center_of_mass(), n8m2_plga.center_of_mass(), 
                                                   box=n8_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n8pplinker[count] = mda.analysis.distances.distance_array(n8pl_link.center_of_mass(), n8peg_link.center_of_mass(), 
                                                   box=n8_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n8peg[count] = mda.analysis.distances.distance_array(n8l1_plgapeg.center_of_mass(), n8l2_plgapeg.center_of_mass(), 
                                                   box=n8_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn8dmso = np.zeros(shape=(3,2))
COM_ppn8dmso[0,0] = np.mean(cb_n8plga)
COM_ppn8dmso[1,0] = np.mean(cb_n8pplinker)
COM_ppn8dmso[2,0] = np.mean(cb_n8peg)
COM_ppn8dmso[0,1] = np.std(cb_n8plga)
COM_ppn8dmso[1,1] = np.std(cb_n8pplinker)
COM_ppn8dmso[2,1] = np.std(cb_n8peg)
COM_ppn8dmso

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 8 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn8dmso = np.zeros(shape=7)
for i in range(n8_mon-1):
    if i < int((n8_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn8dmso[i] = 0
        elif i != 0:
            blen_ppn8dmso[i] = np.sum(blen_ppn8dmso[i-1]) + COM_ppn8dmso[0,0]    
    elif i == int((n8_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn8dmso[i] = np.sum(blen_ppn8dmso[i-1]) + COM_ppn8dmso[1,0]
    elif i > int((n8_mon/2)-1):
    #    # PEG contour length values
        blen_ppn8dmso[i] = np.sum(blen_ppn8dmso[i-1]) + COM_ppn8dmso[2,0]
        
blen_ppn8dmso

In [ ]:
# Error prop. into natural log std deviation
mk_n8ppdmso = cor_n8plgapeg_dmso[1]/cor_n8plgapeg_dmso[0]
mk_n8ppdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n8_blksplgapeg_dmso , n8plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n8dmso, n8_plgapeg_dmso, blen_ppn8dmso, 4, 0 , 36001)

In [ ]:
n8_blksplgapeg_dmso

In [ ]:
n8plgapeg_lpdmso

In [ ]:
n8plgapeg_lpdmso[2]

In [ ]:
np.mean(n8plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_ppn8dmso

In [ ]:
gg_n8plgapeg_dmso = line_fit(np.mean(n8plgapeg_lpdmso[2]),blen_ppn8dmso)

In [ ]:
gg_n6plgapeg_n8dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn8dmso)

In [ ]:
gg_n8plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 8 PLGA-PEG in DMSO

In [ ]:
blppn8dmso = []
for i in range(n8_mon-1):
    if i < int((n8_mon/2)-1):
        # PLGA contour length values
        blppn8dmso.append(COM_ppn8dmso[0,0])    
    elif i == int((n8_mon/2)-1):
        #PLGA-PEG linker length
        blppn8dmso.append(COM_ppn8dmso[1,0]) 
    elif i > int((n8_mon/2)-1):
    #    # PEG contour length values
        blppn8dmso.append(COM_ppn8dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn8dmso = np.sum(blppn8dmso)
lc_ppn8dmso

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso[1,1] = lc_ppn8dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn8dmso = np.mean(n8_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn8dmso = np.mean(n8_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,1] = a_ppn8dmso/b_ppn8dmso
re2e_rgsq_plgapegdmso

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso[1,1] = lc_ppn8dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn8dmso = np.mean(n8_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn8dmso = np.mean(n8_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,1] = aa_ppn8dmso/bb_ppn8dmso
rhy_rg_plgapegdmso

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,1] = np.mean(n8_blksplgapeg_dmso["Avg persistence length"])/lc_ppn8dmso
kuhnlpL_plgapegdmso

#### Block averaged Radius of gyration and persistence length, N = 8 PLGA-PEG in DMSO

In [ ]:
np.mean(n8_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n8_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n8_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n8_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
             capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn8dmso, gg_n6plgapeg_n8dmso, color='teal')
plt.plot(blen_ppn8dmso, gg_n8plgapeg_dmso, color='dodgerblue')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,30)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 33.7 $\AA$ ± 2.04 $\AA$', fontsize=15, color='dodgerblue')

In [ ]:
rgplgapeg_olig_dmso[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG DMSO"] = n8_blksplgapeg_dmso["Avg Radius of gyration"]
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso[r"$L_{p}$ [Angstrom] N = 8 PLGA-PEG DMSO"] = n8_blksplgapeg_dmso["Avg persistence length"]
pers_plgapegt_dmso

### N = 10 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n10_plgapeg_dmso = mda.Universe("n10plga_peg_dmso/n10plgapeg_DMSO.pdb", "n10plga_peg_dmso/NoDMSO_n10plgapeg.xtc")

In [ ]:
n10_plgapeg_dmso.trajectory

In [ ]:
len(n10_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n10dmso = n10_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n10dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n10plgapeg_dmso = pers_length(plgapeg_n10dmso,10)
crv_n10plgapeg_dmso

### Orientation Order parameter vs. time 

In [ ]:
n10_mon = 10 
start = 0
end = 36001

In [ ]:
OOp_n10plgapeg_dmso = orientation_order_param(plgapeg_n10dmso, n10_plgapeg_dmso, n10_mon, start, end)

In [ ]:
OOp_n10plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_dmso,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_dmso,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 10 PLGA-PEG in DMSO

In [ ]:
OOp_n10plgapeg_dmso[82]

In [ ]:
OOp_n10plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n10plgapeg_dmso)

In [ ]:
np.std(OOp_n10plgapeg_dmso)

### Mean squared end to end distance 

In [ ]:
eVec_n10plgapeg_dmso, e2edis_n10plgapeg_dmso = mean_sq_e2e(plgapeg_n10dmso, n10_plgapeg_dmso, n10_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_dmso[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_dmso[1]/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,5)

#### Ree Autocorrelation function

In [ ]:
e2edis_n10plgapeg_dmso[1]

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn10plgapegdmso_Ree, eqvar_n10plgapegdmso = obs_autocorr_RA(e2edis_n10plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegdmso_Ree

In [ ]:
eqvar_n10plgapegdmso

In [ ]:
ACn10plgapegdmso_sqRee, evsqRee_n10plgapegdmso = obs_autocorr_RA(e2edis_n10plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n10plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Ree[1]/100, ACn8plgapegdmso_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Ree[1]/100, ACn10plgapegdmso_Ree[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn10dmso_rh = hydro_rad_poly(plgapeg_n10dmso, n10_plgapeg_dmso, n10_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8dmso_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10dmso_rh/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
t_corr = 10000
window_shift = 9

In [ ]:
ACn10plgapegdmsoRh, eqvarRh_n10plgapegdmso = obs_autocorr_RA(plgapegn10dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegdmsoRh

In [ ]:
eqvarRh_n10plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmsoRh[1]/100, ACn8plgapegdmsoRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmsoRh[1]/100, ACn10plgapegdmsoRh[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO','N = 8 in DMSO','N = 10 in DMSO'], frameon=False, fontsize=14)
plt.xlim(0,50)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n10_mon = 10 
start = 0
end = 36001
t_corr = 1000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn10dmso, tcSUM_plgapegn10dmso = pos_bead_autocorr_RA(plgapeg_n10dmso, n10_plgapeg_dmso, n10_mon, t_corr, window_shift
                                                                 , start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn10dmso

In [ ]:
tcSUM_plgapegn10dmso.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn10dmso = tcRA_plgapegn10dmso[1]/100
ydata_plgapegn10dmso = tcRA_plgapegn10dmso[0]

In [ ]:
ydata_plgapegn10dmso.shape

In [ ]:
xdata_plgapegn10dmso.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n10_mon = 10
n_b = n10_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn10dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 9
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n10dmso = Parameters()
#ppfit_n10dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n10dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn10dmso = Minimizer(res_plgapegn10dmso, ppfit_n10dmso, fcn_args=(xdata_plgapegn10dmso, ydata_plgapegn10dmso))
out_ppn10dmso = mini_ppn10dmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(out_ppn10dmso.params)

In [ ]:
out_ppn10dmso.params

In [ ]:
# with h varying, tr = 5.05535808 +/- 0.03753700, h* = 0.00761718 +/- 0.01288101
tdmso_n10plgapeg = []
n_bonds = 9
for i in range(len(xdata_plgapegn10dmso)):
    tdmso_n10plgapeg.append(zimm_relax_fit(xdata_plgapegn10dmso[i], 4.76278747, 0.26,  n_bonds))


In [ ]:
t_c, tz_n10plgapeg, zimm_n10plgapeg, rouse_n10plgapeg = zimm_relax_func(1, 4.76278747, 0.26,  n_bonds)

In [ ]:
len(tz_n10plgapeg)

In [ ]:
x_pos_n6 = np.arange(5)
x_pos_n8 = np.arange(7)
x_pos_n10 = np.arange(9)
width = 0.25
aa_types_n10 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO', fontsize=18)
plt.bar(x_pos_n6+(2*width), tz_n6plgapeg, width, ecolor='black',capsize=5, color='teal', label="N = 6")
plt.bar(x_pos_n8+(width), tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.bar(x_pos_n10, tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10')
plt.xticks(x_pos_n10, labels=aa_types_n10, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n10 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO N = 10', fontsize=18)
plt.bar(x_pos_n10, rouse_n10plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n10+width, zimm_n10plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n10+width,labels=aa_eigen_n10, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8dmso, tdmso_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10dmso, tdmso_n10plgapeg, color='darkmagenta')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen10 = np.array([ydata_plgapegn10dmso - np.array(tdmso_n10plgapeg)])
rppdmso_msen10

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, rppdmso_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, rppdmso_msen10, color='darkmagenta', s=s_n10)
plt.title(r'Relaxation time Fitting Residuals PLGA in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,100])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 10 PLGA-PEG in DMSO

In [ ]:
n10plgapeg_rgens_dmso, cor_n10plgapeg_dmso, N10plgapeg_cos_dmso, rgdmso_n10plgapeg = get_rg_pers_poly(plgapeg_n10dmso
                                                                                                    , n10_plgapeg_dmso,start,end)

In [ ]:
n10plgapeg_rgens_dmso[0].shape

In [ ]:
cor_n10plgapeg_dmso[3]

In [ ]:
N10plgapeg_cos_dmso

In [ ]:
rgdmso_n10plgapeg

In [ ]:
n10plgapeg_rgens_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_dmso[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_dmso[0]/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO', 'N = 10 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
t_corr = 10000
window_shift = 9

In [ ]:
ACn10plgapegdmso_Rg, eqvarRg_n10plgapegdmso = obs_autocorr_RA(n10plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegdmso_Rg

In [ ]:
eqvarRg_n10plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Rg[1]/100, ACn8plgapegdmso_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Rg[1]/100, ACn10plgapegdmso_Rg[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n10plga = np.zeros(shape=(36001))
cb_n10pplinker = np.zeros(shape=(36001))
cb_n10peg = np.zeros(shape=(36001))
count = 0
for ts in n10_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n10m1_plga = n10_plgapeg_dmso.select_atoms("resid 1")
    n10m2_plga = n10_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n10pl_link = n10_plgapeg_dmso.select_atoms("resid 5")
    n10peg_link = n10_plgapeg_dmso.select_atoms("resid 6")
    # last two PEG residues 
    n10l1_plgapeg = n10_plgapeg_dmso.select_atoms("resid 9")
    n10l2_plgapeg = n10_plgapeg_dmso.select_atoms("resid 10")
    
    cb_n10plga[count] = mda.analysis.distances.distance_array(n10m1_plga.center_of_mass(), n10m2_plga.center_of_mass(), 
                                                   box=n10_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n10pplinker[count] = mda.analysis.distances.distance_array(n10pl_link.center_of_mass(), n10peg_link.center_of_mass(), 
                                                   box=n10_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n10peg[count] = mda.analysis.distances.distance_array(n10l1_plgapeg.center_of_mass(), n10l2_plgapeg.center_of_mass(), 
                                                   box=n10_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn10dmso = np.zeros(shape=(3,2))
COM_ppn10dmso[0,0] = np.mean(cb_n10plga)
COM_ppn10dmso[1,0] = np.mean(cb_n10pplinker)
COM_ppn10dmso[2,0] = np.mean(cb_n10peg)
COM_ppn10dmso[0,1] = np.std(cb_n10plga)
COM_ppn10dmso[1,1] = np.std(cb_n10pplinker)
COM_ppn10dmso[2,1] = np.std(cb_n10peg)
COM_ppn10dmso

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 10 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn10dmso = np.zeros(shape=9)
for i in range(n10_mon-1):
    if i < int((n10_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn10dmso[i] = 0
        elif i != 0:
            blen_ppn10dmso[i] = np.sum(blen_ppn10dmso[i-1]) + COM_ppn10dmso[0,0]    
    elif i == int((n10_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn10dmso[i] = np.sum(blen_ppn10dmso[i-1]) + COM_ppn10dmso[1,0]
    elif i > int((n10_mon/2)-1):
    #    # PEG contour length values
        blen_ppn10dmso[i] = np.sum(blen_ppn10dmso[i-1]) + COM_ppn10dmso[2,0]
        
blen_ppn10dmso

In [ ]:
# Error prop. into natural log std deviation
mk_n10ppdmso = cor_n10plgapeg_dmso[1]/cor_n10plgapeg_dmso[0]
mk_n10ppdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn10dmso, np.log(cor_n10plgapeg_dmso[0]), yerr=mk_n10ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n10_blksplgapeg_dmso , n10plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n10dmso, n10_plgapeg_dmso, blen_ppn10dmso, 5, 0 , 36001)

In [ ]:
n10_blksplgapeg_dmso

In [ ]:
n10plgapeg_lpdmso

In [ ]:
n10plgapeg_lpdmso[2]

In [ ]:
np.mean(n10plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_ppn10dmso

In [ ]:
gg_n10plgapeg_dmso = line_fit(np.mean(n10plgapeg_lpdmso[2]),blen_ppn10dmso)

In [ ]:
gg_n8plgapeg_n10dmso = line_fit(np.mean(n8plgapeg_lpdmso[2]),blen_ppn10dmso)

In [ ]:
gg_n6plgapeg_n10dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn10dmso)

In [ ]:
gg_n10plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 10 PLGA-PEG in DMSO

In [ ]:
blppn10dmso = []
for i in range(n10_mon-1):
    if i < int((n10_mon/2)-1):
        # PLGA contour length values
        blppn10dmso.append(COM_ppn10dmso[0,0])    
    elif i == int((n10_mon/2)-1):
        #PLGA-PEG linker length
        blppn10dmso.append(COM_ppn10dmso[1,0]) 
    elif i > int((n10_mon/2)-1):
    #    # PEG contour length values
        blppn10dmso.append(COM_ppn10dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn10dmso = np.sum(blppn10dmso)
lc_ppn10dmso

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso[1,2] = lc_ppn10dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn10dmso = np.mean(n10_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn10dmso = np.mean(n10_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,2] = a_ppn10dmso/b_ppn10dmso
re2e_rgsq_plgapegdmso

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso[1,2] = lc_ppn10dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn10dmso = np.mean(n10_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn10dmso = np.mean(n10_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,2] = aa_ppn10dmso/bb_ppn10dmso
rhy_rg_plgapegdmso

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,2] = np.mean(n10_blksplgapeg_dmso["Avg persistence length"])/lc_ppn10dmso
kuhnlpL_plgapegdmso

#### Block averaged Radius of gyration and persistence length, N = 10 PLGA-PEG in DMSO

In [ ]:
np.mean(n10_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n10_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n10_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n10_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn10dmso, np.log(cor_n10plgapeg_dmso[0]), yerr=mk_n10ppdmso, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn10dmso, gg_n6plgapeg_n10dmso, color='teal')
plt.plot(blen_ppn10dmso, gg_n8plgapeg_n10dmso, color='dodgerblue')
plt.plot(blen_ppn10dmso, gg_n10plgapeg_dmso, color='darkmagenta')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,40)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 33.7 $\AA$ ± 2.04 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -5.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 35.3 $\AA$ ± 0.95 $\AA$', fontsize=15, color='darkmagenta')

In [ ]:
rgplgapeg_olig_dmso[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG DMSO"] = n10_blksplgapeg_dmso["Avg Radius of gyration"]
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso[r"$L_{p}$ [Angstrom] N = 10 PLGA-PEG DMSO"] = n10_blksplgapeg_dmso["Avg persistence length"]
pers_plgapegt_dmso

### N = 16 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n16_plgapeg_dmso = mda.Universe("n16plga_peg_dmso/n16plgapeg_DMSO.pdb", "n16plga_peg_dmso/NoDMSO_n16plgapeg.xtc")

In [ ]:
n16_plgapeg_dmso.trajectory

In [ ]:
len(n16_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n16dmso = n16_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n16dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n16plgapeg_dmso = pers_length(plgapeg_n16dmso,16)
crv_n16plgapeg_dmso

#### Orientation Order parameter vs. time 

In [ ]:
n16_mon = 16 
start = 0
end = 36001

In [ ]:
OOp_n16plgapeg_dmso = orientation_order_param(plgapeg_n16dmso, n16_plgapeg_dmso, n16_mon, start, end)

In [ ]:
OOp_n16plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_dmso,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_dmso,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_dmso,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 16 PLGA-PEG in DMSO

In [ ]:
OOp_n16plgapeg_dmso[82]

In [ ]:
OOp_n16plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n16plgapeg_dmso)

In [ ]:
np.std(OOp_n16plgapeg_dmso)

#### Mean squared end to end distance 

In [ ]:
eVec_n16plgapeg_dmso, e2edis_n16plgapeg_dmso = mean_sq_e2e(plgapeg_n16dmso, n16_plgapeg_dmso, n16_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_dmso[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_dmso[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_dmso[1]/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Ree Autocorrelation function

In [ ]:
e2edis_n16plgapeg_dmso[1]

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn16plgapegdmso_Ree, eqvar_n16plgapegdmso = obs_autocorr_RA(e2edis_n16plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegdmso_Ree

In [ ]:
eqvar_n16plgapegdmso

In [ ]:
ACn16plgapegdmso_sqRee, evsqRee_n16plgapegdmso = obs_autocorr_RA(e2edis_n16plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n16plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Ree[1]/100, ACn8plgapegdmso_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Ree[1]/100, ACn10plgapegdmso_Ree[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmso_Ree[1]/100, ACn16plgapegdmso_Ree[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 16 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
#plt.grid(b=True)
plt.xlim(0,50)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn16dmso_rh = hydro_rad_poly(plgapeg_n16dmso, n16_plgapeg_dmso, n16_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8dmso_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10dmso_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16dmso_rh/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn16plgapegdmsoRh, eqvarRh_n16plgapegdmso = obs_autocorr_RA(plgapegn16dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegdmsoRh

In [ ]:
eqvarRh_n16plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmsoRh[1]/100, ACn8plgapegdmsoRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmsoRh[1]/100, ACn10plgapegdmsoRh[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmsoRh[1]/100, ACn16plgapegdmsoRh[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO','N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO'], frameon=False, fontsize=14)
plt.xlim(0,50)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n16_mon = 16 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn16dmso, tcSUM_plgapegn16dmso = pos_bead_autocorr_RA(plgapeg_n16dmso, n16_plgapeg_dmso, n16_mon, t_corr, window_shift
                                                                 , start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn16dmso

In [ ]:
tcSUM_plgapegn16dmso.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn16dmso = tcRA_plgapegn16dmso[1]/100
ydata_plgapegn16dmso = tcRA_plgapegn16dmso[0]

In [ ]:
ydata_plgapegn16dmso.shape

In [ ]:
xdata_plgapegn16dmso.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16)
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n16_mon = 16
n_b = n16_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn16dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 15
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n16dmso = Parameters()
#ppfit_n16dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n16dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn16dmso = Minimizer(res_plgapegn16dmso, ppfit_n16dmso, fcn_args=(xdata_plgapegn16dmso, ydata_plgapegn16dmso))
out_ppn16dmso = mini_ppn16dmso.leastsq()
#bfit_n16 = ydata_n16dmso + out_n16dmso.residual 

In [ ]:
report_fit(out_ppn16dmso.params)

In [ ]:
out_ppn16dmso.params

In [ ]:
# with h varying, tr = 7.83893332 +/- 0.03945703, h* = 0.25999623 +/- 0.01294047
tdmso_n16plgapeg = []
n_bonds = 15
for i in range(len(xdata_plgapegn16dmso)):
    tdmso_n16plgapeg.append(zimm_relax_fit(xdata_plgapegn16dmso[i], 7.84547549, 0.26,  n_bonds))


In [ ]:
t_c, tz_n16plgapeg, zimm_n16plgapeg, rouse_n16plgapeg = zimm_relax_func(1, 7.84547549, 0.26,  n_bonds)

In [ ]:
len(tz_n16plgapeg)

In [ ]:
x_pos_n8 = np.arange(7)
x_pos_n10 = np.arange(9)
x_pos_n16 = np.arange(15)
width = 0.25
aa_types_n16 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$",r"$\tau_{11}$",r"$\tau_{12}$",r"$\tau_{13}$",r"$\tau_{14}$",r"$\tau_{15}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO', fontsize=18)
plt.bar(x_pos_n8+(2*width), tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.bar(x_pos_n10+(width), tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10')
plt.bar(x_pos_n16, tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16')
plt.xticks(x_pos_n16, labels=aa_types_n16, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n16 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$",r"$a_{11}$",r"$a_{12}$",r"$a_{13}$",r"$a_{14}$",r"$a_{15}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO N = 16', fontsize=18)
plt.bar(x_pos_n16, rouse_n16plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n16+width, zimm_n16plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n16+width,labels=aa_eigen_n16, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16)
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8dmso, tdmso_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10dmso, tdmso_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16dmso, tdmso_n16plgapeg, color='maroon')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen16 = np.array([ydata_plgapegn16dmso - np.array(tdmso_n16plgapeg)])
#rppdmso_msen16

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, rppdmso_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, rppdmso_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, rppdmso_msen16, color='maroon', s=s_n16)
plt.title(r'Relaxation time Fitting Residuals PLGA in DMSO', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,150])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 16 PLGA-PEG in DMSO

In [ ]:
# Key variables 
n16_mon = 16 
start = 0
end = 36001

In [ ]:
n16plgapeg_rgens_dmso, cor_n16plgapeg_dmso, n16plgapeg_cos_dmso, rgdmso_n16plgapeg = get_rg_pers_poly(plgapeg_n16dmso
                                                                                                      , n16_plgapeg_dmso,start,end)

In [ ]:
n16plgapeg_rgens_dmso[0].shape

In [ ]:
cor_n16plgapeg_dmso[3]

In [ ]:
n16plgapeg_cos_dmso

In [ ]:
rgdmso_n16plgapeg

In [ ]:
n16plgapeg_rgens_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_dmso[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_dmso[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_dmso[0]/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,7)

#### Rg Autocorrelation

In [ ]:
ACn16plgapegdmso_Rg, eqvarRg_n16plgapegdmso = obs_autocorr_RA(n16plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegdmso_Rg

In [ ]:
eqvarRg_n16plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Rg[1]/100, ACn8plgapegdmso_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Rg[1]/100, ACn10plgapegdmso_Rg[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmso_Rg[1]/100, ACn16plgapegdmso_Rg[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 16 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-2,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n16plga = np.zeros(shape=(36001))
cb_n16pplinker = np.zeros(shape=(36001))
cb_n16peg = np.zeros(shape=(36001))
count = 0
for ts in n16_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n16m1_plga = n16_plgapeg_dmso.select_atoms("resid 1")
    n16m2_plga = n16_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n16pl_link = n16_plgapeg_dmso.select_atoms("resid 8")
    n16peg_link = n16_plgapeg_dmso.select_atoms("resid 9")
    # last two PEG residues 
    n16l1_plgapeg = n16_plgapeg_dmso.select_atoms("resid 15")
    n16l2_plgapeg = n16_plgapeg_dmso.select_atoms("resid 16")
    
    cb_n16plga[count] = mda.analysis.distances.distance_array(n16m1_plga.center_of_mass(), n16m2_plga.center_of_mass(), 
                                                   box=n16_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n16pplinker[count] = mda.analysis.distances.distance_array(n16pl_link.center_of_mass(), n16peg_link.center_of_mass(), 
                                                   box=n16_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n16peg[count] = mda.analysis.distances.distance_array(n16l1_plgapeg.center_of_mass(), n16l2_plgapeg.center_of_mass(), 
                                                   box=n16_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn16dmso = np.zeros(shape=(3,2))
COM_ppn16dmso[0,0] = np.mean(cb_n16plga)
COM_ppn16dmso[1,0] = np.mean(cb_n16pplinker)
COM_ppn16dmso[2,0] = np.mean(cb_n16peg)
COM_ppn16dmso[0,1] = np.std(cb_n16plga)
COM_ppn16dmso[1,1] = np.std(cb_n16pplinker)
COM_ppn16dmso[2,1] = np.std(cb_n16peg)
COM_ppn16dmso

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 16 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn16dmso = np.zeros(shape=15)
for i in range(n16_mon-1):
    if i < int((n16_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn16dmso[i] = 0
        elif i != 0:
            blen_ppn16dmso[i] = np.sum(blen_ppn16dmso[i-1]) + COM_ppn16dmso[0,0]    
    elif i == int((n16_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn16dmso[i] = np.sum(blen_ppn16dmso[i-1]) + COM_ppn16dmso[1,0]
    elif i > int((n16_mon/2)-1):
    #    # PEG contour length values
        blen_ppn16dmso[i] = np.sum(blen_ppn16dmso[i-1]) + COM_ppn16dmso[2,0]
        
blen_ppn16dmso

In [ ]:
# Error prop. into natural log std deviation
mk_n16ppdmso = cor_n16plgapeg_dmso[1]/cor_n16plgapeg_dmso[0]
mk_n16ppdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn16dmso, np.log(cor_n16plgapeg_dmso[0]), yerr=mk_n16ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n16_blksplgapeg_dmso , n16plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n16dmso, n16_plgapeg_dmso, blen_ppn16dmso, 5, 0 , 36001)

In [ ]:
n16_blksplgapeg_dmso

In [ ]:
n16plgapeg_lpdmso

In [ ]:
n16plgapeg_lpdmso[2]

In [ ]:
np.mean(n16plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_ppn16dmso

In [ ]:
gg_n16plgapeg_dmso = line_fit(np.mean(n16plgapeg_lpdmso[2]),blen_ppn16dmso)

In [ ]:
gg_n10plgapeg_n16dmso = line_fit(np.mean(n10plgapeg_lpdmso[2]),blen_ppn16dmso)

In [ ]:
gg_n8plgapeg_n16dmso = line_fit(np.mean(n8plgapeg_lpdmso[2]),blen_ppn16dmso)

In [ ]:
gg_n6plgapeg_n16dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn16dmso)

In [ ]:
gg_n16plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 16 PLGA-PEG in DMSO

In [ ]:
blppn16dmso = []
for i in range(n16_mon-1):
    if i < int((n16_mon/2)-1):
        # PLGA contour length values
        blppn16dmso.append(COM_ppn16dmso[0,0])    
    elif i == int((n16_mon/2)-1):
        #PLGA-PEG linker length
        blppn16dmso.append(COM_ppn16dmso[1,0]) 
    elif i > int((n16_mon/2)-1):
    #    # PEG contour length values
        blppn16dmso.append(COM_ppn16dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn16dmso = np.sum(blppn16dmso)
lc_ppn16dmso

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso[1,3] = lc_ppn16dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn16dmso = np.mean(n16_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn16dmso = np.mean(n16_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,3] = a_ppn16dmso/b_ppn16dmso
re2e_rgsq_plgapegdmso

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso[1,3] = lc_ppn16dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn16dmso = np.mean(n16_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn16dmso = np.mean(n16_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,3] = aa_ppn16dmso/bb_ppn16dmso
rhy_rg_plgapegdmso

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,3] = np.mean(n16_blksplgapeg_dmso["Avg persistence length"])/lc_ppn16dmso
kuhnlpL_plgapegdmso

#### Block averaged Radius of gyration and persistence length, N = 16 PLGA-PEG in DMSO

In [ ]:
np.mean(n16_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n16_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n16_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n16_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn10dmso, np.log(cor_n10plgapeg_dmso[0]), yerr=mk_n10ppdmso, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn16dmso, np.log(cor_n16plgapeg_dmso[0]), yerr=mk_n16ppdmso, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn16dmso, gg_n6plgapeg_n16dmso, color='teal')
plt.plot(blen_ppn16dmso, gg_n8plgapeg_n16dmso, color='dodgerblue')
plt.plot(blen_ppn16dmso, gg_n10plgapeg_n16dmso, color='darkmagenta')
plt.plot(blen_ppn16dmso, gg_n16plgapeg_dmso, color='maroon')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,70)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 33.7 $\AA$ ± 2.04 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -5.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 35.3 $\AA$ ± 0.95 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -4.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 35.3 $\AA$ ± 3.17 $\AA$', fontsize=15, color='maroon')

In [ ]:
rgplgapeg_olig_dmso[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG DMSO"] = n16_blksplgapeg_dmso["Avg Radius of gyration"]
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso[r"$L_{p}$ [Angstrom] N = 16 PLGA-PEG DMSO"] = n16_blksplgapeg_dmso["Avg persistence length"]
pers_plgapegt_dmso

### N = 20 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n20_plgapeg_dmso = mda.Universe("n20plga_plgapeg_dmso/n20plgapeg_DMSO.pdb", "n20plga_plgapeg_dmso/NoDMSO_n20plgapeg.xtc")

In [ ]:
n20_plgapeg_dmso.trajectory

In [ ]:
len(n20_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n20dmso = n20_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n20dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n20plgapeg_dmso = pers_length(plgapeg_n20dmso,20)
crv_n20plgapeg_dmso

#### Orientation Order parameter vs. time 

In [ ]:
n20_mon = 20
start = 0
end = 36001

In [ ]:
OOp_n20plgapeg_dmso = orientation_order_param(plgapeg_n20dmso, n20_plgapeg_dmso, n20_mon, start, end)

In [ ]:
OOp_n20plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_dmso,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_dmso,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_dmso,linewidth=2, color='maroon')
plt.plot(trj_len/100, OOp_n20plgapeg_dmso,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 20 PLGA-PEG in DMSO

In [ ]:
OOp_n20plgapeg_dmso[82]

In [ ]:
OOp_n20plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n20plgapeg_dmso)

In [ ]:
np.std(OOp_n20plgapeg_dmso)

#### Mean squared end to end distance 

In [ ]:
eVec_n20plgapeg_dmso, e2edis_n20plgapeg_dmso = mean_sq_e2e(plgapeg_n20dmso, n20_plgapeg_dmso, n20_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_dmso[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_dmso[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_dmso[1]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, e2edis_n20plgapeg_dmso[1]/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,4)

#### Ree Autocorrelation function

In [ ]:
e2edis_n20plgapeg_dmso[1]

In [ ]:
t_corr = 4000
window_shift = 6

In [ ]:
ACn20plgapegdmso_Ree, eqvar_n120pegdmso = obs_autocorr_RA(e2edis_n20plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegdmso_Ree

In [ ]:
eqvar_n16plgapegdmso

In [ ]:
ACn20plgapegdmso_sqRee, evsqRee_n20plgapegdmso = obs_autocorr_RA(e2edis_n20plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n20plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Ree[1]/100, ACn8plgapegdmso_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Ree[1]/100, ACn10plgapegdmso_Ree[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmso_Ree[1]/100, ACn16plgapegdmso_Ree[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegdmso_Ree[1]/100, ACn20plgapegdmso_Ree[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 15 Ree','N = 20 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn20dmso_rh = hydro_rad_poly(plgapeg_n20dmso, n20_plgapeg_dmso, n20_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8dmso_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10dmso_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16dmso_rh/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, plgapegn20dmso_rh/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,6)

#### Rh autocorrelation

In [ ]:
ACn20plgapegdmsoRh, eqvarRh_n20plgapegdmso = obs_autocorr_RA(plgapegn20dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegdmsoRh

In [ ]:
eqvarRh_n20plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmsoRh[1]/100, ACn8plgapegdmsoRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmsoRh[1]/100, ACn10plgapegdmsoRh[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmsoRh[1]/100, ACn16plgapegdmsoRh[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegdmsoRh[1]/100, ACn20plgapegdmsoRh[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO','N = 8 in DMSO','N = 10 in DMSO','N = 15 in DMSO','N = 20 in DMSO'], frameon=False, fontsize=14)
plt.xlim(0,50)
#plt.ylim(-2,1.5)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n20_mon = 20 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn20dmso, tcSUM_plgapegn20dmso = pos_bead_autocorr_RA(plgapeg_n20dmso, n20_plgapeg_dmso, n20_mon, t_corr, window_shift
                                                                 , start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn20dmso

In [ ]:
tcSUM_plgapegn20dmso.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn20dmso = tcRA_plgapegn20dmso[1]/100
ydata_plgapegn20dmso = tcRA_plgapegn20dmso[0]

In [ ]:
ydata_plgapegn20dmso.shape

In [ ]:
xdata_plgapegn20dmso.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20dmso, ydata_plgapegn20dmso, color='orangered', s=s_n20)
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n20_mon = 20
n_b = n20_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn20dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 19
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n20dmso = Parameters()
#ppfit_n20dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n20dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn20dmso = Minimizer(res_plgapegn20dmso, ppfit_n20dmso, fcn_args=(xdata_plgapegn20dmso, ydata_plgapegn20dmso))
out_ppn20dmso = mini_ppn20dmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(out_ppn20dmso.params)

In [ ]:
out_ppn20dmso.params

In [ ]:
# with h varying, tr = 9.95886003 +/- 0.09664351, h* = 0.25999717 +/- 0.04971985
tdmso_n20plgapeg = []
n_bonds = 19
for i in range(len(xdata_plgapegn20dmso)):
    tdmso_n20plgapeg.append(zimm_relax_fit(xdata_plgapegn20dmso[i], 9.98045843, 0.26,  n_bonds))


In [ ]:
t_c, tz_n20plgapeg, zimm_n20plgapeg, rouse_n20plgapeg = zimm_relax_func(1, 9.98045843, 0.26,  n_bonds)

In [ ]:
x_pos_n10 = np.arange(9)
x_pos_n16 = np.arange(15)
x_pos_n20 = np.arange(19)
width = 0.25
aa_types_n20 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$", r"$\tau_{11}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{15}$",r"$\tau_{16}$",r"$\tau_{17}$", r"$\tau_{18}$", r"$\tau_{19}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.bar(x_pos_n10+(2*width), tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10', align='center')
plt.bar(x_pos_n16+width, tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16')
plt.bar(x_pos_n20, tz_n20plgapeg, width, ecolor='black',capsize=5, color='orangered', label='N = 20', align='center')
plt.xticks(x_pos_n20, labels=aa_types_n20, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n20 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$", r"$a_{11}$", r"$a_{12}$", r"$a_{13}$", r"$a_{14}$",r"$a_{15}$",r"$a_{16}$",r"$a_{17}$",
           r"$a_{18}$",r"$a_{19}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in DMSO N = 20', fontsize=18)
plt.bar(x_pos_n20, rouse_n20plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n20+width, zimm_n20plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n20+width,labels=aa_eigen_n20, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20dmso, ydata_plgapegn20dmso, color='orangered', s=s_n20)
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8dmso, tdmso_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10dmso, tdmso_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16dmso, tdmso_n16plgapeg, color='maroon')
plt.plot(xdata_plgapegn20dmso, tdmso_n20plgapeg, color='orangered')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen20 = np.array([ydata_plgapegn20dmso - np.array(tdmso_n20plgapeg)])
rppdmso_msen20

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, rppdmso_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, rppdmso_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, rppdmso_msen16, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20dmso, rppdmso_msen20, color='orangered', s=s_n20)
plt.title(r'Relaxation time Fitting Residuals PLGA in DMSO', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,250])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 20 PLGA-PEG in DMSO

In [ ]:
# Key variables 
n20_mon = 20
start = 0
end = 36001

In [ ]:
n20plgapeg_rgens_dmso, cor_n20plgapeg_dmso, n20plgapeg_cos_dmso, rgdmso_n20plgapeg = get_rg_pers_poly(plgapeg_n20dmso
                                                                                                      ,n20_plgapeg_dmso,start,end)

In [ ]:
n20plgapeg_rgens_dmso[0].shape

In [ ]:
cor_n20plgapeg_dmso[3]

In [ ]:
n20plgapeg_cos_dmso

In [ ]:
rgdmso_n20plgapeg

In [ ]:
n20plgapeg_rgens_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_dmso[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_dmso[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_dmso[0]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, n20plgapeg_rgens_dmso[0]/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
ACn20plgapegdmso_Rg, eqvarRg_n20plgapegdmso = obs_autocorr_RA(n20plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegdmso_Rg

In [ ]:
eqvarRg_n20plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegdmso_Rg[1]/100, ACn8plgapegdmso_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegdmso_Rg[1]/100, ACn10plgapegdmso_Rg[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegdmso_Rg[1]/100, ACn16plgapegdmso_Rg[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegdmso_Rg[1]/100, ACn20plgapegdmso_Rg[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 15 Rg','N = 20 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-2,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n20plga = np.zeros(shape=(36001))
cb_n20pplinker = np.zeros(shape=(36001))
cb_n20peg = np.zeros(shape=(36001))
count = 0
for ts in n20_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n20m1_plga = n20_plgapeg_dmso.select_atoms("resid 1")
    n20m2_plga = n20_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n20pl_link = n20_plgapeg_dmso.select_atoms("resid 10")
    n20peg_link = n20_plgapeg_dmso.select_atoms("resid 11")
    # last two PEG residues 
    n20l1_plgapeg = n20_plgapeg_dmso.select_atoms("resid 19")
    n20l2_plgapeg = n20_plgapeg_dmso.select_atoms("resid 20")
    
    cb_n20plga[count] = mda.analysis.distances.distance_array(n20m1_plga.center_of_mass(), n20m2_plga.center_of_mass(), 
                                                   box=n20_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n20pplinker[count] = mda.analysis.distances.distance_array(n20pl_link.center_of_mass(), n20peg_link.center_of_mass(), 
                                                   box=n20_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n20peg[count] = mda.analysis.distances.distance_array(n20l1_plgapeg.center_of_mass(), n20l2_plgapeg.center_of_mass(), 
                                                   box=n20_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn20dmso = np.zeros(shape=(3,2))
COM_ppn20dmso[0,0] = np.mean(cb_n20plga)
COM_ppn20dmso[1,0] = np.mean(cb_n20pplinker)
COM_ppn20dmso[2,0] = np.mean(cb_n20peg)
COM_ppn20dmso[0,1] = np.std(cb_n20plga)
COM_ppn20dmso[1,1] = np.std(cb_n20pplinker)
COM_ppn20dmso[2,1] = np.std(cb_n20peg)
COM_ppn20dmso

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 20 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn20dmso = np.zeros(shape=19)
for i in range(n20_mon-1):
    if i < int((n20_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn20dmso[i] = 0
        elif i != 0:
            blen_ppn20dmso[i] = np.sum(blen_ppn20dmso[i-1]) + COM_ppn20dmso[0,0]    
    elif i == int((n20_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn20dmso[i] = np.sum(blen_ppn20dmso[i-1]) + COM_ppn20dmso[1,0]
    elif i > int((n20_mon/2)-1):
    #    # PEG contour length values
        blen_ppn20dmso[i] = np.sum(blen_ppn20dmso[i-1]) + COM_ppn20dmso[2,0]
        
blen_ppn20dmso

In [ ]:
# Error prop. into natural log std deviation
mk_n20ppdmso = cor_n20plgapeg_dmso[1]/cor_n20plgapeg_dmso[0]
mk_n20ppdmso

In [ ]:
cor_n20plgapeg_dmso[0]

In [ ]:
cor_n20plgapeg_dmso[1]

In [ ]:
a = np.arange(29)
p1 = np.ma.masked_where(a == 25, np.log(cor_n30plgapeg_dmso[0]))
costh_n30dmso = np.ma.masked_where(a == 26, p1)
e1 = np.ma.masked_where(a == 25, mk_n30ppdmso)
errln_n30dmso  = np.ma.masked_where(a == 26, e1)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn20dmso, cor_n20plgapeg_dmso[0], yerr=cor_n20plgapeg_dmso[1], color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n20_blksplgapeg_dmso , n20plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n20dmso, n20_plgapeg_dmso, blen_ppn20dmso, 5, 0 , 36001)

In [ ]:
n20_blksplgapeg_dmso

In [ ]:
n20plgapeg_lpdmso

In [ ]:
n20plgapeg_lpdmso[2]

In [ ]:
np.mean(n20plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_ppn20dmso

In [ ]:
gg_n20plgapeg_dmso = line_fit(np.mean(n20plgapeg_lpdmso[2]),blen_ppn20dmso)

In [ ]:
gg_n16plgapeg_n20dmso = line_fit(np.mean(n16plgapeg_lpdmso[2]),blen_ppn20dmso)

In [ ]:
gg_n10plgapeg_n20dmso = line_fit(np.mean(n10plgapeg_lpdmso[2]),blen_ppn20dmso)

In [ ]:
gg_n8plgapeg_n20dmso = line_fit(np.mean(n8plgapeg_lpdmso[2]),blen_ppn20dmso)

In [ ]:
gg_n6plgapeg_n20dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn20dmso)

In [ ]:
gg_n20plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 20 PLGA-PEG in DMSO

In [ ]:
blppn20dmso = []
for i in range(n20_mon-1):
    if i < int((n20_mon/2)-1):
        # PLGA contour length values
        blppn20dmso.append(COM_ppn20dmso[0,0])    
    elif i == int((n20_mon/2)-1):
        #PLGA-PEG linker length
        blppn20dmso.append(COM_ppn20dmso[1,0]) 
    elif i > int((n20_mon/2)-1):
    #    # PEG contour length values
        blppn20dmso.append(COM_ppn20dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn20dmso = np.sum(blppn20dmso)
lc_ppn20dmso

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso[1,4] = lc_ppn20dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn20dmso = np.mean(n20_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn20dmso = np.mean(n20_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,4] = a_ppn20dmso/b_ppn20dmso
re2e_rgsq_plgapegdmso

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso[1,4] = lc_ppn20dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn20dmso = np.mean(n20_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn20dmso = np.mean(n20_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,4] = aa_ppn20dmso/bb_ppn20dmso
rhy_rg_plgapegdmso

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,4] = np.mean(n20_blksplgapeg_dmso["Avg persistence length"])/lc_ppn20dmso
kuhnlpL_plgapegdmso

#### Block averaged Radius of gyration and persistence length, N = 20 PLGA-PEG in DMSO

In [ ]:
np.mean(n20_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n20_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n20_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n20_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
cor_n20plgapeg_dmso[0][:17]

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn10dmso, np.log(cor_n10plgapeg_dmso[0]), yerr=mk_n10ppdmso, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn16dmso, np.log(cor_n16plgapeg_dmso[0]), yerr=mk_n16ppdmso, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn20dmso[:17], np.log(cor_n20plgapeg_dmso[0][:17]), yerr=mk_n20ppdmso[:17], color='orangered', linestyle="None"
        ,marker='o',capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn20dmso, gg_n6plgapeg_n20dmso, color='teal')
plt.plot(blen_ppn20dmso, gg_n8plgapeg_n20dmso, color='dodgerblue')
plt.plot(blen_ppn20dmso, gg_n10plgapeg_n20dmso, color='darkmagenta')
plt.plot(blen_ppn20dmso, gg_n16plgapeg_n20dmso, color='maroon')
plt.plot(blen_ppn20dmso[:17], gg_n20plgapeg_dmso[:17], color='orangered')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-7,1.5)
plt.xlim(0,90)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.57,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 33.7 $\AA$ ± 2.04 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -5.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 35.3 $\AA$ ± 0.95 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -4.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 35.3 $\AA$ ± 3.17 $\AA$', fontsize=15, color='maroon')
plt.text(0.5, -4.50,r'$N_{PLGA-PEG}$ = 20: $L_{p}$ = 36.6 $\AA$ ± 3.45 $\AA$', fontsize=15, color='orangered')

In [ ]:
rgplgapeg_olig_dmso[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG DMSO"] = n20_blksplgapeg_dmso["Avg Radius of gyration"]
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso[r"$L_{p}$ [Angstrom] N = 20 PLGA-PEG DMSO"] = n20_blksplgapeg_dmso["Avg persistence length"]
pers_plgapegt_dmso

### N = 30 PLGA-PEG/DMSO 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n30_plgapeg_dmso = mda.Universe("n30plga_plgapeg_dmso/n30plgapeg_DMSO.pdb", "n30plga_plgapeg_dmso/NoDMSO_n30plgapeg.xtc")

In [ ]:
n30_plgapeg_dmso.trajectory

In [ ]:
len(n30_plgapeg_dmso.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n30dmso = n30_plgapeg_dmso.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n30dmso.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n30plgapeg_dmso = pers_length(plgapeg_n30dmso,30)
crv_n30plgapeg_dmso

#### Orientation Order parameter vs. time 

In [ ]:
n30_mon = 30
start = 0
end = 36001

In [ ]:
OOp_n30plgapeg_dmso = orientation_order_param(plgapeg_n30dmso, n30_plgapeg_dmso, n30_mon, start, end)

In [ ]:
OOp_n30plgapeg_dmso.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_dmso,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_dmso,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_dmso,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_dmso,linewidth=2, color='maroon')
plt.plot(trj_len/100, OOp_n20plgapeg_dmso,linewidth=2, color='orangered')
plt.plot(trj_len/100, OOp_n30plgapeg_dmso,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO','N = 30 in DMSO'],
           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-2,2)

#### Mean Orientation Order Parameter for N = 30 PLGA-PEG in DMSO

In [ ]:
OOp_n30plgapeg_dmso[82]

In [ ]:
OOp_n30plgapeg_dmso[239]

In [ ]:
np.mean(OOp_n30plgapeg_dmso)

In [ ]:
np.std(OOp_n30plgapeg_dmso)

#### Mean squared end to end distance 

In [ ]:
eVec_n30plgapeg_dmso, e2edis_n30plgapeg_dmso = mean_sq_e2e(plgapeg_n30dmso, n30_plgapeg_dmso, n30_mon, start, end)

In [ ]:
np.save('Reetime_N6ppdmso.npy', e2edis_n6plgapeg_dmso)
np.save('Reetime_N8ppdmso.npy', e2edis_n8plgapeg_dmso)
np.save('Reetime_N10ppdmso.npy', e2edis_n10plgapeg_dmso)
np.save('Reetime_N16ppdmso.npy', e2edis_n16plgapeg_dmso)
#np.save('Reetime_N20ppdmso.npy', e2edis_n20plgapeg_dmso)
#np.save('Reetime_N30ppdmso.npy', e2edis_n30plgapeg_dmso)

In [ ]:
e2edis_n6plgapeg_dmso = np.load('poly_strut_data/Reetime_N6ppdmso.npy', allow_pickle=True)
e2edis_n8plgapeg_dmso = np.load('poly_strut_data/Reetime_N8ppdmso.npy', allow_pickle=True)
e2edis_n10plgapeg_dmso = np.load('poly_strut_data/Reetime_N10ppdmso.npy', allow_pickle=True)
e2edis_n16plgapeg_dmso = np.load('poly_strut_data/Reetime_N16ppdmso.npy', allow_pickle=True)
e2edis_n20plgapeg_dmso = np.load('poly_strut_data/Reetime_N20ppdmso.npy', allow_pickle=True)
e2edis_n30plgapeg_dmso = np.load('poly_strut_data/Reetime_N30ppdmso.npy', allow_pickle=True)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, e2edis_n6plgapeg_dmso[1]/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, e2edis_n8plgapeg_dmso[1]/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, e2edis_n10plgapeg_dmso[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_dmso[1]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, e2edis_n20plgapeg_dmso[1]/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, e2edis_n30plgapeg_dmso[1]/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in DMSO', 'N = 20 in DMSO','N = 30 in DMSO'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO','N = 30 in DMSO'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,12)

#### Ree Autocorrelation function

In [ ]:
e2edis_n30plgapeg_dmso[1]

In [ ]:
t_corr = 3000
window_shift = 5

In [ ]:
ACn30plgapegdmso_Ree, eqvar_n30plgapegdmso = obs_autocorr_RA(e2edis_n30plgapeg_dmso[1],t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegdmso_Ree

In [ ]:
eqvar_n30plgapegdmso

In [ ]:
ACn30plgapegdmso_sqRee, evsqRee_n30plgapegdmso = obs_autocorr_RA(e2edis_n30plgapeg_dmso[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n30plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Ree[1]/100, ACn6plgapegdmso_Ree[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmso_Ree[1]/100, ACn8plgapegdmso_Ree[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegdmso_Ree[1]/100, ACn10plgapegdmso_Ree[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegdmso_Ree[1]/100, ACn16plgapegdmso_Ree[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegdmso_Ree[1]/100, ACn20plgapegdmso_Ree[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegdmso_Ree[1]/100, ACn30plgapegdmso_Ree[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 16 Ree','N = 20 Ree','N = 30 Ree'], frameon=False, fontsize=14)
#plt.legend(['N = 6 Lee','N = 8 Lee','N = 10 Lee','N = 15 Lee'], frameon=False, fontsize=14)
#plt.legend(['N = 20 Lee','N = 30 Lee'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn30dmso_rh = hydro_rad_poly(plgapeg_n30dmso, n30_plgapeg_dmso, n30_mon, start, end)

In [ ]:
plgapegn6dmso_rh = np.load('poly_strut_data/Rhydtime_N6ppdmso.npy', allow_pickle=True)
plgapegn8dmso_rh = np.load('poly_strut_data/Rhydtime_N8ppdmso.npy', allow_pickle=True)
plgapegn10dmso_rh = np.load('poly_strut_data/Rhydtime_N10ppdmso.npy', allow_pickle=True)
plgapegn16dmso_rh = np.load('poly_strut_data/Rhydtime_N16ppdmso.npy', allow_pickle=True)
plgapegn20dmso_rh = np.load('poly_strut_data/Rhydtime_N20ppdmso.npy', allow_pickle=True)
plgapegn30dmso_rh = np.load('poly_strut_data/Rhydtime_N30ppdmso.npy', allow_pickle=True)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, plgapegn6dmso_rh/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, plgapegn8dmso_rh/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, plgapegn10dmso_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16dmso_rh/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, plgapegn20dmso_rh/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, plgapegn30dmso_rh/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in DMSO', 'N = 20 in DMSO','N = 30 in DMSO'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO','N = 30 in DMSO'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(1,6)

In [ ]:
np.save('Rhydtime_N6ppdmso.npy', plgapegn6dmso_rh)
np.save('Rhydtime_N8ppdmso.npy', plgapegn8dmso_rh)
np.save('Rhydtime_N10ppdmso.npy', plgapegn10dmso_rh)
np.save('Rhydtime_N16ppdmso.npy', plgapegn16dmso_rh)
np.save('Rhydtime_N20ppdmso.npy', plgapegn20dmso_rh)
np.save('Rhydtime_N30ppdmso.npy', plgapegn30dmso_rh)

#### Rh autocorrelation

In [ ]:
ACn30plgapegdmsoRh, eqvarRh_n30plgapegdmso = obs_autocorr_RA(plgapegn30dmso_rh,t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegdmsoRh

In [ ]:
eqvarRh_n30plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmsoRh[1]/100, ACn6plgapegdmsoRh[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmsoRh[1]/100, ACn8plgapegdmsoRh[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegdmsoRh[1]/100, ACn10plgapegdmsoRh[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegdmsoRh[1]/100, ACn16plgapegdmsoRh[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegdmsoRh[1]/100, ACn20plgapegdmsoRh[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegdmsoRh[1]/100, ACn30plgapegdmsoRh[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend([r'N = 6 $R_{hyd}$','N = 8 $R_{hyd}$','N = 10 $R_{hyd}$','N = 16 $R_{hyd}$','N = 20 $R_{hyd}$','N = 30 $R_{hyd}$']
 ,frameon=False, fontsize=14)
#plt.legend([r'N = 6 $R_{hyd}$','N = 8 $R_{hyd}$','N = 10 $R_{hyd}$','N = 15 $R_{hyd}$'], frameon=False, fontsize=14)
#plt.legend([r'N = 20 $R_{hyd}$',r'N = 30 $R_{hyd}$'], frameon=False, fontsize=14)
plt.xlim(0,50)
plt.ylim(-2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n30_mon = 30 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn30dmso, tcSUM_plgapegn30dmso = pos_bead_autocorr_RA(plgapeg_n30dmso, n30_plgapeg_dmso, n30_mon, t_corr, window_shift
                                                                 , start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn30dmso

In [ ]:
tcSUM_plgapegn30dmso.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn30dmso = tcRA_plgapegn30dmso[1]/100
ydata_plgapegn30dmso = tcRA_plgapegn30dmso[0]

In [ ]:
ydata_plgapegn30dmso.shape

In [ ]:
xdata_plgapegn30dmso.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6, label=r'$N_{PLGA-PEG}$ = 6')
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8, label=r'$N_{PLGA-PEG}$ = 8')
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10, label=r'$N_{PLGA-PEG}$ = 10')
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16, label=r'$N_{PLGA-PEG}$ = 16')
plt.scatter(xdata_plgapegn20dmso, ydata_plgapegn20dmso, color='orangered', s=s_n20, label=r'$N_{PLGA-PEG}$ = 20')
plt.scatter(xdata_plgapegn30dmso, ydata_plgapegn30dmso, color='darkolivegreen', s=s_n30, label=r'$N_{PLGA-PEG}$ = 30')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
plt.legend(fontsize=14, frameon=False,loc='upper right')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
np.save('n6plgapeg_dmsoCt_RA.npy', tcRA_plgapegn6dmso)
np.save('n8plgapeg_dmsoCt_RA.npy', tcRA_plgapegn8dmso)
np.save('n10plgapeg_dmsoCt_RA.npy', tcRA_plgapegn10dmso)
np.save('n10plgapeg_dmsoCt_RA.npy', tcRA_plgapegn16dmso)
np.save('n20plgapeg_dmsoCt_RA.npy', tcRA_plgapegn20dmso)
np.save('n30plgapeg_dmsoCt_RA.npy', tcRA_plgapegn30dmso)

In [ ]:
ts = 1
hs = 0.1
n30_mon = 30
n_b = n30_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
#zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn30dmso(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 29
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n30dmso = Parameters()
#ppfit_n30dmso.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n30dmso.add(name='t_first', value=2)

In [ ]:
mini_ppn30dmso = Minimizer(res_plgapegn30dmso, ppfit_n30dmso, fcn_args=(xdata_plgapegn30dmso, ydata_plgapegn30dmso))
out_ppn30dmso = mini_ppn30dmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(out_ppn30dmso.params)

In [ ]:
out_ppn30dmso.params

In [ ]:
# with h varying, tr = 39.6126855 +/- 0.40000367, h* = 1.4685e-05 +/- 0.00475622
tdmso_n30plgapeg = []
n_bonds = 29
for i in range(len(xdata_plgapegn30dmso)):
    tdmso_n30plgapeg.append(zimm_relax_fit(xdata_plgapegn30dmso[i], 33.8204049, 0.26,  n_bonds))


In [ ]:
t_c, tz_n30plgapeg, zimm_n30plgapeg, rouse_n30plgapeg = zimm_relax_func(1, 33.8204049, 0.26,  n_bonds)

In [ ]:
x_pos_n16 = np.arange(15)
x_pos_n20 = np.arange(19)
x_pos_n30 = np.arange(29)
#x_pos_n30 = np.arange(29)
width = 0.25
aa_types_n30 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$", r"$\tau_{11}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{15}$",r"$\tau_{16}$",r"$\tau_{17}$", r"$\tau_{18}$", r"$\tau_{19}$",r"$\tau_{20}$", r"$\tau_{21}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{25}$",r"$\tau_{26}$",r"$\tau_{27}$", r"$\tau_{28}$", r"$\tau_{29}$"]
fig = plt.figure(figsize=(10,10))
plt.title('PLGA-PEG in DMSO', fontsize=18)
plt.bar(x_pos_n16+(2*width), tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16', align='center')
plt.bar(x_pos_n20+(width), tz_n20plgapeg, width, ecolor='black',capsize=5, color='orangered', label='N = 20', align='center')
plt.bar(x_pos_n30, tz_n30plgapeg, width, ecolor='black',capsize=5, color='darkolivegreen', label='N = 30', align='center')
plt.xticks(x_pos_n30, labels=aa_types_n30, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.autoscale(enable=True, axis='both')
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
x_pos = np.arange(29)
aa_eigen_n30 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$", r"$a_{11}$", r"$a_{12}$", r"$a_{13}$", r"$a_{14}$",r"$a_{15}$",r"$a_{16}$",r"$a_{17}$",
           r"$a_{18}$",r"$a_{19}$",r"$a_{20}$", r"$a_{21}$", r"$a_{22}$", r"$a_{23}$", r"$a_{24}$",r"$a_{25}$",r"$a_{26}$",
            r"$a_{27}$",r"$a_{28}$",r"$a_{29}$"]
fig = plt.figure(figsize=(10,10))
plt.title('PLGA-PEG in DMSO N = 30', fontsize=18)
plt.bar(x_pos_n30, rouse_n30plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n30+width, zimm_n30plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n30+width,labels=aa_eigen_n30, fontsize=12)
plt.autoscale(enable=True, axis='both')
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, ydata_plgapegn6dmso, color='teal', s=s_n6, label=r'$N_{PLGA-PEG}$ = 6')
plt.scatter(xdata_plgapegn8dmso, ydata_plgapegn8dmso, color='dodgerblue', s=s_n8, label=r'$N_{PLGA-PEG}$ = 8')
plt.scatter(xdata_plgapegn10dmso, ydata_plgapegn10dmso, color='darkmagenta', s=s_n10, label=r'$N_{PLGA-PEG}$ = 10')
plt.scatter(xdata_plgapegn16dmso, ydata_plgapegn16dmso, color='maroon', s=s_n16, label=r'$N_{PLGA-PEG}$ = 16')
plt.scatter(xdata_plgapegn20dmso, ydata_plgapegn20dmso, color='orangered', s=s_n20, label=r'$N_{PLGA-PEG}$ = 20')
plt.scatter(xdata_plgapegn30dmso, ydata_plgapegn30dmso, color='darkolivegreen', s=s_n30, label=r'$N_{PLGA-PEG}$ = 30')
plt.plot(xdata_plgapegn6dmso, tdmso_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8dmso, tdmso_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10dmso, tdmso_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16dmso, tdmso_n16plgapeg, color='maroon')
plt.plot(xdata_plgapegn20dmso, tdmso_n20plgapeg, color='orangered')
plt.plot(xdata_plgapegn30dmso, tdmso_n30plgapeg, color='darkolivegreen')
plt.title(r'Positional bead autocorrelation PLGA-PEG in DMSO', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppdmso_msen30 = np.array([ydata_plgapegn30dmso- np.array(tdmso_n30plgapeg)])
#rpdmso_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6dmso, rppdmso_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8dmso, rppdmso_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10dmso, rppdmso_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16dmso, rppdmso_msen16, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20dmso, rppdmso_msen20, color='orangered', s=s_n20)
plt.scatter(xdata_plgapegn30dmso, rppdmso_msen30, color='darkolivegreen', s=s_n30)
plt.title(r'Relaxation time Fitting Residuals PEG in DMSO', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,260])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Relaxation Times Alpha Exponent

In [ ]:
out_ppn6dmso.params

In [ ]:
out_ppn8dmso.params

In [ ]:
out_ppn10dmso.params

In [ ]:
out_ppn16dmso.params

In [ ]:
out_ppn20dmso.params

In [ ]:
out_ppn30dmso.params

In [ ]:
zimm_t1_plgapegdmso = np.zeros(shape=(5,6))
zimm_t1_plgapegdmso[3] = np.array([6,8,10,16,20,30])
zimm_t1_plgapegdmso[0] = np.array([1.69732007, 2.59078779, 4.76278747, 7.84547549, 9.98045843, 33.8204049])
zimm_t1_plgapegdmso[1] = np.array([0.01076820, 0.01467422, 0.03009827, 0.03171532, 0.07771197, 0.30859742])
zimm_t1_plgapegdmso

In [ ]:
np.log10(zimm_t1_plgapegdmso[0])

In [ ]:
np.log10(zimm_t1_plgapegdmso[3])

In [ ]:
zimm_t1_plgapegdmso[1]

In [ ]:
#Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
t1plgapegdmso_logerr = np.zeros(shape=(6))
for i in range(6):
    t1plgapegdmso_logerr[i] = 0.434*((zimm_t1_plgapegdmso[1][i])/(zimm_t1_plgapegdmso[0][i]))
    
t1plgapegdmso_logerr

In [ ]:
zimm_t1_plgapegdmso[2] = t1plgapegdmso_logerr

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(zimm_t1_plgapegdmso[3]), np.log10(zimm_t1_plgapegdmso[0]), yerr=zimm_t1_plgapegdmso[2], color='#0371EC', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
#plt.plot(np.log10(n_plgapeg), gg_dmso, color='#A58262')

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_alplgapegdmso = LinearRegression(fit_intercept=True)
model_alplgapegdmso.fit(np.log10(zimm_t1_plgapegdmso[3]).reshape(-1,1), np.log10(zimm_t1_plgapegdmso[0]))
# Slope here is in nanometers
print("Model slope:    ", model_alplgapegdmso.coef_[0])
print("Model intercept:", model_alplgapegdmso.intercept_)

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
pair_regression(np.log10(zimm_t1_plgapegdmso[3]),np.log10(zimm_t1_plgapegdmso[0]))

In [ ]:
ggal_plgapegdmso = model_alplgapegdmso.predict(np.log10(zimm_t1_plgapegdmso[3]).reshape(-1,1))
ggal_plgapegdmso

In [ ]:
np.log10(zimm_t1_plgapegdmso[0])

In [ ]:
# Residuals between the true y data and model y data 
resid_alpdmso = np.log10(zimm_t1_plgapegdmso[0]) - ggal_plgapegdmso
resid_alpdmso

In [ ]:
# How to calculate mean squared error, N - 2 is dof 
mse_alpdmso = np.sum(resid_alpdmso**2)/(len(resid_alpdmso) - 2)
mse_alpdmso

In [ ]:
# Normalized root mean squared error
np.sqrt(mse_alpdmso)/(np.max(np.log10(zimm_t1_plgapegdmso[0])) - np.min(np.log10(zimm_t1_plgapegdmso[0])))

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(np.log10(zimm_t1_plgapegdmso[0]), ggal_plgapegdmso))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
getPairStats(np.log10(zimm_t1_plgapegdmso[3]),np.log10(zimm_t1_plgapegdmso[0]))

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttplgapegdmso = np.log10(zimm_t1_plgapegdmso[3]) - np.mean(np.log10(zimm_t1_plgapegdmso[3]))
nhui_plgapegdmso = nt_ttplgapegdmso**2
np.sum(nhui_plgapegdmso)

In [ ]:
# How to calculate Sxy
ata_tr = np.sum((np.log10(zimm_t1_plgapegdmso[0]) - np.mean(np.log10(zimm_t1_plgapegdmso[0])))*(nt_ttplgapegdmso))
ata_tr

In [ ]:
# Test statistic, t0
slp_esttr = ata_tr/np.sum(nhui_plgapegdmso)
slp_esttr/np.sqrt(mse_alpdmso/(np.sum(nhui_plgapegdmso)))

In [ ]:
# t-value with 95 % confidence intervals (alpha/2, n-2)
# Value of test statistic is greater than threshold, reject H0
n = len(resid_alpdmso) 
alpha = 0.05
scipy.stats.t.ppf(1-alpha/2, n - 2)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_trpdmso = scipy.stats.t.ppf(1-alpha/2, n - 2)*np.sqrt(mse_alpdmso/np.sum(nhui_plgapegdmso))
flc_trpdmso

In [ ]:
# standard error of the slope
np.sqrt(mse_alpdmso/np.sum(nhui_plgapegdmso))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
simple_regression_conf(np.log10(zimm_t1_plgapegdmso[3]),np.log10(zimm_t1_plgapegdmso[0]), alpha=0.05)

In [ ]:
zimm_t1_plgapegdmso[4] = ggal_plgapegdmso
zimm_t1_plgapegdmso

In [ ]:
np.save('PLGAPEGdmso_relax_times.npy', zimm_t1_plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(zimm_t1_plgapegdmso[3]), np.log10(zimm_t1_plgapegdmso[0]), yerr=zimm_t1_plgapegdmso[2], color='#0371EC', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(np.log10(zimm_t1_plgapegdmso[3]), zimm_t1_plgapegdmso[4], color='#0371EC')
plt.title(r'Relaxation Times Scaling Exponent, PLGA-PEG in DMSO', fontsize=15)
plt.xlabel(r'Log($N_{PLGA-PEG}$)', fontsize=15)
plt.ylabel(r'Log($\tau_{1}$)', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(0.9, 1.4, r'$\alpha_{DMSO}$ = 1.73 ± 0.25', fontsize=15, color='#0371EC')
plt.text(0.9, 1.32, r'$R^{2}$ = 0.976, NRMSE = 0.073', fontsize=15, color='#0371EC')

#### Radius of Gyration vs. time N = 30 PLGA-PEG in DMSO

In [ ]:
# Key variables 
n30_mon = 30
start = 0
end =36001

In [ ]:
n30plgapeg_rgens_dmso, cor_n30plgapeg_dmso, n30plgapeg_cos_dmso, rgdmso_n30plgapeg = get_rg_pers_poly(plgapeg_n30dmso, n30_plgapeg_dmso,
                                                                                                  start,end)

In [ ]:
n30plgapeg_rgens_dmso[0].shape

In [ ]:
rgdmso_n30plgapeg

In [ ]:
n30plgapeg_rgens_dmso.shape

In [ ]:
n6plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N6ppdmso.npy', allow_pickle=True)
n8plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N8ppdmso.npy', allow_pickle=True)
n10plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N10ppdmso.npy', allow_pickle=True)
n16plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N16ppdmso.npy', allow_pickle=True)
n20plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N20ppdmso.npy', allow_pickle=True)
n30plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N30ppdmso.npy', allow_pickle=True)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, n6plgapeg_rgens_dmso[0]/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, n8plgapeg_rgens_dmso[0]/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, n10plgapeg_rgens_dmso[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_dmso[0]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, n20plgapeg_rgens_dmso[0]/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, n30plgapeg_rgens_dmso[0]/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in DMSO', 'N = 20 in DMSO','N = 30 in DMSO'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in DMSO', 'N = 8 in DMSO','N = 10 in DMSO','N = 16 in DMSO','N = 20 in DMSO','N = 30 in DMSO'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in DMSO', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,4)

In [ ]:
np.save('Rgtime_N6ppdmso.npy', n6plgapeg_rgens_dmso)
np.save('Rgtime_N8ppdmso.npy', n8plgapeg_rgens_dmso)
np.save('Rgtime_N10ppdmso.npy', n10plgapeg_rgens_dmso)
np.save('Rgtime_N16ppdmso.npy', n16plgapeg_rgens_dmso)
np.save('Rgtime_N20ppdmso.npy', n20plgapeg_rgens_dmso)
np.save('Rgtime_N30ppdmso.npy', n30plgapeg_rgens_dmso)

In [ ]:
n30plgapeg_rgens_dmso = np.load('poly_strut_data/Rgtime_N30ppdmso.npy', allow_pickle=True)
plgapegn30dmso_rh = np.load('poly_strut_data/Rhydtime_N30ppdmso.npy', allow_pickle=True)

In [ ]:
rg_rh_n30plgapegdmso = n30plgapeg_rgens_dmso[0]/plgapegn30dmso_rh

In [ ]:
trj_len = np.arange(rg_rh_n30plgapegdmso.shape[0])
trj_len.shape

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(trj_len/100, rg_rh_n30plgapegdmso)
plt.xlim(0,360)

In [ ]:
rhy_rg_plgapegdmso = np.load('PLGAPEGDMSO_RgRh_Lc.npy', allow_pickle=True)
rhy_rg_plgapegdmso[0]

In [ ]:
rhy_rg_plgapegdmso[0][5]

In [ ]:
rg_rh_n30plgapegdmso[6402]

In [ ]:
np.where(rg_rh_n30plgapegdmso < rhy_rg_plgapegdmso[0][5])

#### Rg Autocorrelation

In [ ]:
ACn30plgapegdmso_Rg, eqvarRg_n30plgapegdmso = obs_autocorr_RA(n30plgapeg_rgens_dmso[0],t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegdmso_Rg

In [ ]:
eqvarRg_n30plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in DMSO autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegdmso_Rg[1]/100, ACn6plgapegdmso_Rg[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegdmso_Rg[1]/100, ACn8plgapegdmso_Rg[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegdmso_Rg[1]/100, ACn10plgapegdmso_Rg[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegdmso_Rg[1]/100, ACn16plgapegdmso_Rg[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegdmso_Rg[1]/100, ACn20plgapegdmso_Rg[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegdmso_Rg[1]/100, ACn30plgapegdmso_Rg[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 16 Rg','N = 20 Rg','N = 30 Rg'], frameon=False, fontsize=14)
#plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 15 Rg'], frameon=False, fontsize=14)
#plt.legend(['N = 20 Rg','N = 30 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,50)
plt.ylim(-2,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n30plga = np.zeros(shape=(36001))
cb_n30pplinker = np.zeros(shape=(36001))
cb_n30peg = np.zeros(shape=(36001))
count = 0
for ts in n30_plgapeg_dmso.trajectory[0:36001]:
    # 1st two PLGA residues
    n30m1_plga = n30_plgapeg_dmso.select_atoms("resid 1")
    n30m2_plga = n30_plgapeg_dmso.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n30pl_link = n30_plgapeg_dmso.select_atoms("resid 15")
    n30peg_link = n30_plgapeg_dmso.select_atoms("resid 16")
    # last two PEG residues 
    n30l1_plgapeg = n30_plgapeg_dmso.select_atoms("resid 29")
    n30l2_plgapeg = n30_plgapeg_dmso.select_atoms("resid 30")
    
    cb_n30plga[count] = mda.analysis.distances.distance_array(n30m1_plga.center_of_mass(), n30m2_plga.center_of_mass(), 
                                                   box=n30_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n30pplinker[count] = mda.analysis.distances.distance_array(n30pl_link.center_of_mass(), n30peg_link.center_of_mass(), 
                                                   box=n30_plgapeg_dmso.trajectory.ts.dimensions)
    
    cb_n30peg[count] = mda.analysis.distances.distance_array(n30l1_plgapeg.center_of_mass(), n30l2_plgapeg.center_of_mass(), 
                                                   box=n30_plgapeg_dmso.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_ppn30dmso = np.zeros(shape=(3,2))
COM_ppn30dmso[0,0] = np.mean(cb_n30plga)
COM_ppn30dmso[1,0] = np.mean(cb_n30pplinker)
COM_ppn30dmso[2,0] = np.mean(cb_n30peg)
COM_ppn30dmso[0,1] = np.std(cb_n30plga)
COM_ppn30dmso[1,1] = np.std(cb_n30pplinker)
COM_ppn30dmso[2,1] = np.std(cb_n30peg)
COM_ppn30dmso

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 30 PLGA-PEG/DMSO

In [ ]:
# x values
blen_ppn30dmso = np.zeros(shape=29)
for i in range(n30_mon-1):
    if i < int((n30_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_ppn30dmso[i] = 0
        elif i != 0:
            blen_ppn30dmso[i] = np.sum(blen_ppn30dmso[i-1]) + COM_ppn30dmso[0,0]    
    elif i == int((n30_mon/2)-1):
        #PLGA-PEG linker length
        blen_ppn30dmso[i] = np.sum(blen_ppn30dmso[i-1]) + COM_ppn30dmso[1,0]
    elif i > int((n30_mon/2)-1):
    #    # PEG contour length values
        blen_ppn30dmso[i] = np.sum(blen_ppn30dmso[i-1]) + COM_ppn30dmso[2,0]
        
blen_ppn30dmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn30dmso, cor_n30plgapeg_dmso[0], yerr=cor_n30plgapeg_dmso[1], color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# Error prop. into natural log std deviation
mk_n30ppdmso = cor_n30plgapeg_dmso[1]/cor_n30plgapeg_dmso[0]
mk_n30ppdmso

In [ ]:
cor_n30plgapeg_dmso[0]

In [ ]:
cor_n30plgapeg_dmso[1].shape

In [ ]:
a = np.arange(29)
p1 = np.ma.masked_where(a == 25, np.log(cor_n30plgapeg_dmso[0]))
costh_n30dmso = np.ma.masked_where(a == 26, p1)
e1 = np.ma.masked_where(a == 25, mk_n30ppdmso)
errln_n30dmso  = np.ma.masked_where(a == 26, e1)

In [ ]:
mk_n30ppdmso[25] = 0
mk_n30ppdmso[26] = 0
mk_n30ppdmso

In [ ]:
mk_n30ppdmso = cor_n30plgapeg_dmso[1]/cor_n30plgapeg_dmso[0]
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn30dmso, costh_n30dmso, yerr=errln_n30dmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn30dmso, np.log(cor_n30plgapeg_dmso[0]), yerr=mk_n30ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in DMSO
n30_blksplgapeg_dmso , n30plgapeg_lpdmso = bavg_pers_cnt(5, plgapeg_n30dmso, n30_plgapeg_dmso, blen_ppn30dmso, 5, 0 , 36001)

In [ ]:
n30_blksplgapeg_dmso

In [ ]:
np.save("N6blksppdmso.npy", n6_blksplgapeg_dmso)
np.save("N8blksppdmso.npy", n8_blksplgapeg_dmso)
np.save("N10blksppdmso.npy", n10_blksplgapeg_dmso)
np.save("N16blksppdmso.npy", n16_blksplgapeg_dmso)
np.save("N20blksppdmso.npy", n20_blksplgapeg_dmso)
np.save("N30blksppdmso.npy", n30_blksplgapeg_dmso)

In [ ]:
n30plgapeg_lpdmso

In [ ]:
n30plgapeg_lpdmso[2]

In [ ]:
np.mean(n30plgapeg_lpdmso[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_ppn30dmso

In [ ]:
gg_n30plgapeg_dmso = line_fit(np.mean(n30plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n20plgapeg_n30dmso = line_fit(np.mean(n20plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n16plgapeg_n30dmso = line_fit(np.mean(n16plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n10plgapeg_n30dmso = line_fit(np.mean(n10plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n8plgapeg_n30dmso = line_fit(np.mean(n8plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n6plgapeg_n30dmso = line_fit(np.mean(n6plgapeg_lpdmso[2]),blen_ppn30dmso)

In [ ]:
gg_n30plgapeg_dmso

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 30 PLGA-PEG in DMSO

In [ ]:
blppn30dmso = []
for i in range(n30_mon-1):
    if i < int((n30_mon/2)-1):
        # PLGA contour length values
        blppn30dmso.append(COM_ppn30dmso[0,0])    
    elif i == int((n30_mon/2)-1):
        #PLGA-PEG linker length
        blppn30dmso.append(COM_ppn30dmso[1,0]) 
    elif i > int((n30_mon/2)-1):
    #    # PEG contour length values
        blppn30dmso.append(COM_ppn30dmso[2,0]) 

# Contour Length, N = 6 PLGA-PEG in DMSO
lc_ppn30dmso = np.sum(blppn30dmso)
lc_ppn30dmso

#### Mean sq. e2e/Mean sq. Rg vs contour length PLGA-PEG in DMSO

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegdmso[1,5] = lc_ppn30dmso
re2e_rgsq_plgapegdmso

In [ ]:
a_ppn30dmso = np.mean(n30_blksplgapeg_dmso['Avg Sq. end to end distance'])

In [ ]:
b_ppn30dmso = np.mean(n30_blksplgapeg_dmso['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegdmso[0,5] = a_ppn30dmso/b_ppn30dmso
re2e_rgsq_plgapegdmso

#### Error propagation: Mean sq. e2e/Mean sq. Rg

In [ ]:
e2esq_plgapegdmso = np.array([a_ppn6dmso,a_ppn8dmso, a_ppn10dmso, a_ppn16dmso,a_ppn20dmso, a_ppn30dmso])
e2esq_plgapegdmso

In [ ]:
rgsq_plgapegdmso = np.array([b_ppn6dmso,b_ppn8dmso, b_ppn10dmso, b_ppn16dmso, b_ppn20dmso, b_ppn30dmso])
rgsq_plgapegdmso

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stde2e_pdmso = np.array([np.std(n6_blksplgapeg_dmso['Avg Sq. end to end distance']),
                         np.std(n8_blksplgapeg_dmso['Avg Sq. end to end distance']),
                        np.std(n10_blksplgapeg_dmso['Avg Sq. end to end distance']),
                         np.std(n16_blksplgapeg_dmso['Avg Sq. end to end distance']),
                        np.std(n20_blksplgapeg_dmso['Avg Sq. end to end distance']),
                        np.std(n30_blksplgapeg_dmso['Avg Sq. end to end distance'])])
stde2e_pdmso

In [ ]:
stdrgsq_pdmso = np.array([np.std(n6_blksplgapeg_dmso['Avg Sq. radius of gyration']),
                         np.std(n8_blksplgapeg_dmso['Avg Sq. radius of gyration']),
                         np.std(n10_blksplgapeg_dmso['Avg Sq. radius of gyration']),
                          np.std(n16_blksplgapeg_dmso['Avg Sq. radius of gyration']),
                         np.std(n20_blksplgapeg_dmso['Avg Sq. radius of gyration']),
                         np.std(n30_blksplgapeg_dmso['Avg Sq. radius of gyration'])])
stdrgsq_pdmso

In [ ]:
re2e_rgsq_plgapegdmso[0]

In [ ]:
x_pdmso = np.zeros(shape=6)
for i in range(6):
    ad_s = np.sqrt((((stde2e_pdmso[i]/e2esq_plgapegdmso[i])**2) + ((stdrgsq_pdmso[i]/rgsq_plgapegdmso[i])**2)))
    x_pdmso[i] = re2e_rgsq_plgapegdmso[0][i]*ad_s
    
x_pdmso

In [ ]:
re2e_rgsq_plgapegdmso

In [ ]:
re2e_rgsq_plgapegdmso[2] = x_pdmso
re2e_rgsq_plgapegdmso

In [ ]:
re2e_rgsq_plgapegdmso.shape

In [ ]:
np.save('PLGA-PEGdmso_msRee_msRg.npy', re2e_rgsq_plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(re2e_rgsq_plgapegdmso[1]/10, re2e_rgsq_plgapegdmso[0], yerr=re2e_rgsq_plgapegdmso[2], color='#0371EC', linestyle="-",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
plt.title('Mean sq. Ree/Mean sq. Rg, PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>/\left<R_{g}^{2}\right>$', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlim(1,15)
plt.ylim(0,10)

#### Rg/Rh vs contour length PLGA-PEG in DMSO

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegdmso[1,5] = lc_ppn30dmso
rhy_rg_plgapegdmso

In [ ]:
aa_ppn30dmso = np.mean(n30_blksplgapeg_dmso['Avg Radius of gyration'])

In [ ]:
bb_ppn30dmso = np.mean(n30_blksplgapeg_dmso['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegdmso[0,5] = aa_ppn30dmso/bb_ppn30dmso
rhy_rg_plgapegdmso

In [ ]:
rhy_rg_plgapegdmso

#### Error Propagation: Rg/Rh vs contour length

In [ ]:
rgsys_plgapegdmso = np.array([aa_ppn6dmso,aa_ppn8dmso, aa_ppn10dmso, aa_ppn16dmso,aa_ppn20dmso, aa_ppn30dmso])
rgsys_plgapegdmso

In [ ]:
rhydsys_plgapegdmso = np.array([bb_ppn6dmso,bb_ppn8dmso, bb_ppn10dmso, bb_ppn16dmso, bb_ppn20dmso, bb_ppn30dmso])
rhydsys_plgapegdmso

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stdrg_pdmso = np.array([np.std(n6_blksplgapeg_dmso['Avg Radius of gyration']),
                         np.std(n8_blksplgapeg_dmso['Avg Radius of gyration']),
                        np.std(n10_blksplgapeg_dmso['Avg Radius of gyration']),
                        np.std(n16_blksplgapeg_dmso['Avg Radius of gyration']),
                        np.std(n20_blksplgapeg_dmso['Avg Radius of gyration']),
                        np.std(n30_blksplgapeg_dmso['Avg Radius of gyration'])])
stdrg_pdmso

In [ ]:
std_rhydsq_pdmso = np.array([np.std(n6_blksplgapeg_dmso['Avg Hydrodynamic radius']),
                         np.std(n8_blksplgapeg_dmso['Avg Hydrodynamic radius']),
                         np.std(n10_blksplgapeg_dmso['Avg Hydrodynamic radius']),
                          np.std(n16_blksplgapeg_dmso['Avg Hydrodynamic radius']),    
                         np.std(n20_blksplgapeg_dmso['Avg Hydrodynamic radius']),
                         np.std(n30_blksplgapeg_dmso['Avg Hydrodynamic radius'])])
std_rhydsq_pdmso

In [ ]:
xrgrh_pdmso = np.zeros(shape=6)
for i in range(6):
    adn1_s = np.sqrt((((stdrg_pdmso[i]/rgsys_plgapegdmso[i])**2) + ((std_rhydsq_pdmso[i]/rhydsys_plgapegdmso[i])**2)))
    xrgrh_pdmso[i] = rhy_rg_plgapegdmso[0][i]*adn1_s
    
xrgrh_pdmso

In [ ]:
rhy_rg_plgapegdmso

In [ ]:
rhy_rg_plgapegdmso[2] = xrgrh_pdmso
rhy_rg_plgapegdmso

In [ ]:
np.save('PLGAPEGDMSO_RgRh_Lc.npy', rhy_rg_plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(rhy_rg_plgapegdmso[1]/10, rhy_rg_plgapegdmso[0], yerr=rhy_rg_plgapegdmso[2], color='#0371EC', linestyle="-",
             marker='^', capsize=5, capthick=1, ecolor='#0371EC')
plt.title(r'Rg/Rh PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=15)
plt.ylabel(r'$\left<R_{g}\right>/\left<R_{hyd}\right>$', fontsize=15)
plt.xlim(1,15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(0.3,0.7)

### kuhn lenght vs monomer length

In [ ]:
REElclk_plgapegdmso = np.load('PLGAPEGDMSO_REElclk.npy', allow_pickle=True)

In [ ]:
REElclk_plgapegdmso = np.zeros(shape=(4,6))
REElclk_plgapegdmso[0] = e2esq_plgapegdmso
REElclk_plgapegdmso[1] = rhy_rg_plgapegdmso[1]
REElclk_plgapegdmso[2] = stde2e_pdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegdmso[1], REElclk_plgapegdmso[0], color='#0371EC', yerr=REElclk_plgapegdmso[2], linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='#0371EC')

In [ ]:
# Linear fitting procedure: https://www2.isye.gatech.edu/~yxie77/isye2028/lecture12.pdf
ne2e = REElclk_plgapegdmso[1]
from sklearn.linear_model import LinearRegression
model_e2eppdmso = LinearRegression(fit_intercept=True)
model_e2eppdmso.fit(ne2e.reshape(-1,1), REElclk_plgapegdmso[0])
# Slope here is in angstroms
print("Model slope:    ", model_e2eppdmso.coef_[0])
print("Model intercept:", model_e2eppdmso.intercept_)

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
pair_regression(REElclk_plgapegdmso[1],REElclk_plgapegdmso[0])

In [ ]:
gge2e_ppdmso = model_e2eppdmso.predict(ne2e.reshape(-1,1))
gge2e_ppdmso

In [ ]:
REElclk_plgapegdmso[0]

In [ ]:
# Residuals between the true y data and model y data 
resid_e2eppdmso = REElclk_plgapegdmso[0] - gge2e_ppdmso
resid_e2eppdmso

In [ ]:
# How to calculate mean squared error, N - 2 is dof 
mse_e2eppdmso = np.sum(resid_e2eppdmso**2)/(len(resid_e2eppdmso) - 2)
mse_e2eppdmso

In [ ]:
# Normalized root mean squared error
np.sqrt(mse_e2eppdmso)/(np.max(REElclk_plgapegdmso[0]) - np.min(REElclk_plgapegdmso[0]))

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(REElclk_plgapegdmso[0].reshape(-1,1), gge2e_ppdmso))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
getPairStats(REElclk_plgapegdmso[1],REElclk_plgapegdmso[0])

In [ ]:
# How to calculate Sxx
nt_tte2epd = ne2e - np.mean(ne2e)
nhui_e2epd = nt_tte2epd**2
np.sum(nhui_e2epd)

In [ ]:
# How to calculate Sxy
ata = np.sum((REElclk_plgapegdmso[0] - np.mean(REElclk_plgapegdmso[0]))*(ne2e - np.mean(ne2e)))
ata

In [ ]:
# Test statistic, t0
(ata/np.sum(nhui_e2epd))/np.sqrt(mse_e2eppdmso/(np.sum(nhui_e2epd)))

In [ ]:
# t-value with 95 % confidence intervals (alpha, n-2)
# Value of test statistic is greater than threshold, reject H0
n = len(resid_e2eppdmso) 
alpha = 0.05
scipy.stats.t.ppf(1-alpha/2, n - 2)
#scipy.stats.t.ppf(0.95, len(resid_e2epgdmso) - 1)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_e2epd = scipy.stats.t.ppf(1-alpha/2, n - 2)*np.sqrt(mse_e2eppdmso/(np.sum(nhui_e2epd)))
flc_e2epd

In [ ]:
# standard error of the slope
np.sqrt(mse_e2eppdmso/np.sum(nhui_e2epd))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
simple_regression_conf(REElclk_plgapegdmso[1],REElclk_plgapegdmso[0], alpha=0.05)

In [ ]:
REElclk_plgapegdmso[3] = gge2e_ppdmso
REElclk_plgapegdmso

In [ ]:
np.save('PLGAPEGDMSO_REElclk.npy', REElclk_plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegdmso[1], REElclk_plgapegdmso[0], color='#0371EC', yerr=REElclk_plgapegdmso[2], linestyle="None",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
plt.plot(REElclk_plgapegdmso[1], REElclk_plgapegdmso[3], color='#0371EC')
plt.title(r'Kuhn Length, PLGA-PEG in DMSO', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [$\AA$]', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(30, 2000, r'$l^{DMSO}_{k}$ = 12.6 $\AA$ ± 0.56 $\AA$', fontsize=15, color='#0371EC')
plt.text(30, 1860, r'$R^{2}$ = 0.992, NRMSE = 0.037', fontsize=15, color='#0371EC')
plt.xlim(10,150)
plt.ylim(0,2500)

#### Lp alternate method

In [ ]:
REElclk_plgapegdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegdmso[1], REElclk_plgapegdmso[0], color='#0371EC', yerr=REElclk_plgapegdmso[2], linestyle="None",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
#plt.plot(REElclk_plgapegdmso[1]/10, REElclk_plgapegdmso[3], color='#0371EC')
plt.title(r'Kuhn Length, PLGA-PEG in DMSO', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [A]', fontsize=15)
plt.tick_params(labelsize=14)
plt.xlim(10,150)
plt.ylim(0,3000)

In [ ]:
REElclk_plgapegdmso[2]

In [ ]:
REElclk_plgapegdmso[1]

In [ ]:
REElclk_plgapegdmso[0]

In [ ]:
def sqree_lpfitting(lc, lp):
    
    fir_p = 2*lp*lc
    
    sec_p = 1 - ((lp/lc)*(1 - np.exp(-lc/lp)))
    
    return fir_p*sec_p
    

In [ ]:
sqree_lpfitting(16.2, 3.8)

In [ ]:
def perslen_fitplgapegdmso(varbs, xlc, yree):
    fir_varb = varbs['lp']
    test_yree = []
    
    for i in range(len(xlc)):
        model_yree = sqree_lpfitting(xlc[i],fir_varb)
        test_yree.append(yree[i]- model_yree)
        
    return np.array(test_yree)

In [ ]:
from lmfit import Minimizer, Parameters, report_fit

In [ ]:
plgapegdmso_perfit = Parameters()
plgapegdmso_perfit.add(name='lp', value=0)

In [ ]:
perlc_plgapegdmso = Minimizer(perslen_fitplgapegdmso, plgapegdmso_perfit, fcn_args=(REElclk_plgapegdmso[1], REElclk_plgapegdmso[0]))
olplc_plgapegdmso = perlc_plgapegdmso.leastsq()
#bfit_n10 = ydata_n10dmso + out_n10dmso.residual 

In [ ]:
report_fit(olplc_plgapegdmso.params)

In [ ]:
olplc_plgapegdmso.params

In [ ]:
plgapegdmsolcree = []
xlc = REElclk_plgapegdmso[1]
for i in range(len(REElclk_plgapegdmso[1])):
    plgapegdmsolcree.append(sqree_lpfitting(xlc[i], 6.11642961))
    
md_nlppdmso = np.array(plgapegdmsolcree)

In [ ]:
md_nlppdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegdmso[1], REElclk_plgapegdmso[0], color='#0371EC', yerr=REElclk_plgapegdmso[2], linestyle="None",marker='v',
    capsize=5, capthick=1, ecolor='#0371EC')
plt.plot(REElclk_plgapegdmso[1], md_nlppdmso, color='r', ls='--')
plt.title(r'$L_{p}$ alt calc, PLGA-PEG in DMSO', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [$\AA$]', fontsize=15)
plt.legend([r'Model: $L_{p}$ = 6.12 $\AA$ ± 0.16 $\AA$','Sim'], frameon=False, fontsize=14)
plt.tick_params(labelsize=14)
plt.xlim(10,150)
plt.ylim(0,3000)

#### Error Propagation: Lp/Lc ratio

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegdmso[0,5] = np.mean(n30_blksplgapeg_dmso["Avg persistence length"])/lc_ppn30dmso
kuhnlpL_plgapegdmso

In [ ]:
kuhnlpL_plgapegdmso

In [ ]:
lpsys_plgapegdmso = np.array([np.mean(n6_blksplgapeg_dmso["Avg persistence length"]),
                         np.mean(n8_blksplgapeg_dmso["Avg persistence length"]),
                        np.mean(n10_blksplgapeg_dmso["Avg persistence length"]),
                        np.mean(n16_blksplgapeg_dmso["Avg persistence length"]),  
                        np.mean(n20_blksplgapeg_dmso["Avg persistence length"]),
                        np.mean(n30_blksplgapeg_dmso["Avg persistence length"])])
lpsys_plgapegdmso

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stdlp_pdmso = np.array([np.std(n6_blksplgapeg_dmso["Avg persistence length"]),
                         np.std(n8_blksplgapeg_dmso["Avg persistence length"]),
                        np.std(n10_blksplgapeg_dmso["Avg persistence length"]),
                        np.std(n16_blksplgapeg_dmso["Avg persistence length"]),
                        np.std(n20_blksplgapeg_dmso["Avg persistence length"]),
                        np.std(n30_blksplgapeg_dmso["Avg persistence length"])])
stdlp_pdmso

In [ ]:
xlp_pdmso = np.zeros(shape=6)
for i in range(6):
    adn2_s = np.sqrt((((stdlp_pdmso[i]/lpsys_plgapegdmso[i])**2)))
    xlp_pdmso[i] = kuhnlpL_plgapegdmso[0][i]*adn2_s
    
xlp_pdmso

In [ ]:
kuhnlpL_plgapegdmso[2] = xlp_pdmso
kuhnlpL_plgapegdmso

In [ ]:
np.save('PLGAPEGDMSO_lk_LpL.npy', kuhnlpL_plgapegdmso)

In [ ]:
plt.figure(figsize=(7,7))
plt.xlabel(r'$N_{PLGA-PEG}$', fontsize=15)
plt.ylabel(r'$\left<L_{p}\right>/L_{c}$', fontsize=15)
plt.title('Level of rigidity, PLGA-PEG in DMSO', fontsize=15)
plt.errorbar(kuhnlpL_plgapegdmso[1], kuhnlpL_plgapegdmso[0], yerr=kuhnlpL_plgapegdmso[2], marker='^',capsize=5, capthick=1)
plt.tick_params(axis='both',labelsize=15, width=1)
plt.ylim(0,2)
plt.xlim(5,35)

### Block averaged Radius of gyration and persistence length, N = 30 PLGA-PEG in DMSO

In [ ]:
np.mean(n30_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.std(n30_blksplgapeg_dmso["Avg persistence length"])

In [ ]:
np.mean(n30_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
np.std(n30_blksplgapeg_dmso["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, cor_n6plgapeg_dmso[0], yerr=cor_n6plgapeg_dmso[1], color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, cor_n8plgapeg_dmso[0], yerr=cor_n8plgapeg_dmso[1], color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn10dmso, cor_n10plgapeg_dmso[0], yerr=cor_n10plgapeg_dmso[1], color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn16dmso, cor_n16plgapeg_dmso[0], yerr=cor_n16plgapeg_dmso[1], color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn20dmso, cor_n20plgapeg_dmso[0], yerr=cor_n20plgapeg_dmso[1], color='orangered', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn30dmso, cor_n30plgapeg_dmso[0], yerr=cor_n30plgapeg_dmso[1], color='darkolivegreen', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-0.5,1)
plt.xlim(0,130)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
#plt.text(0.5, -6.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 24.7 $\AA$ ± 1.34 $\AA$', fontsize=15, color='teal')
#plt.text(0.5, -6.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 27.3.0 $\AA$ ± 4.09 $\AA$', fontsize=15, color='dodgerblue')
#plt.text(0.5, -6.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 27.5 $\AA$ ± 2.92 $\AA$', fontsize=15, color='darkmagenta')
#plt.text(0.5, -5.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 31.3 $\AA$ ± 2.30 $\AA$', fontsize=15, color='maroon')
#plt.text(0.5, -5.50,r'$N_{PLGA-PEG}$ = 20: $L_{p}$ = 31.6 $\AA$ ± 3.45 $\AA$', fontsize=15, color='orangered')
#plt.text(0.5, -5.10,r'$N_{PLGA-PEG}$ = 30: $L_{p}$ = 28.9 $\AA$ ± 2.25 $\AA$', fontsize=15, color='darkolivegreen')

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_ppn6dmso, np.log(cor_n6plgapeg_dmso[0]), yerr=mk_n6ppdmso, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn8dmso, np.log(cor_n8plgapeg_dmso[0]), yerr=mk_n8ppdmso, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn10dmso, np.log(cor_n10plgapeg_dmso[0]), yerr=mk_n10ppdmso, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn16dmso, np.log(cor_n16plgapeg_dmso[0]), yerr=mk_n16ppdmso, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn20dmso[:16], np.log(cor_n20plgapeg_dmso[0][:16]), yerr=mk_n20ppdmso[:16], color='orangered', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_ppn30dmso, np.log(cor_n30plgapeg_dmso[0]), yerr=mk_n30ppdmso, color='darkolivegreen', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_ppn30dmso, gg_n6plgapeg_n30dmso, color='teal')
plt.plot(blen_ppn30dmso, gg_n8plgapeg_n30dmso, color='dodgerblue')
plt.plot(blen_ppn30dmso, gg_n10plgapeg_n30dmso, color='darkmagenta')
plt.plot(blen_ppn30dmso, gg_n16plgapeg_n30dmso, color='maroon')
plt.plot(blen_ppn30dmso, gg_n20plgapeg_n30dmso, color='orangered')
plt.plot(blen_ppn30dmso, gg_n30plgapeg_dmso, color='darkolivegreen')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in DMSO', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-7,1.5)
plt.xlim(0,130)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -6.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 36.2 $\AA$ ± 3.33 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -6.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 33.7 $\AA$ ± 2.04 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -6.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 35.3 $\AA$ ± 0.95 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -5.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 35.3 $\AA$ ± 3.17 $\AA$', fontsize=15, color='maroon')
plt.text(0.5, -5.50,r'$N_{PLGA-PEG}$ = 20: $L_{p}$ = 36.6 $\AA$ ± 3.45 $\AA$', fontsize=15, color='orangered')
plt.text(0.5, -5.10,r'$N_{PLGA-PEG}$ = 30: $L_{p}$ = 37.5 $\AA$ ± 1.78 $\AA$', fontsize=15, color='darkolivegreen')

In [ ]:
rgplgapeg_olig_dmso[r"$R_{g}$ [Angstrom] N = 30 PEG DMSO "] = n30_blksplgapeg_dmso["Avg Radius of gyration"]
rgplgapeg_olig_dmso

In [ ]:
pers_plgapegt_dmso[r"$L_{p}$ [Angstrom] N = 30 PEG DMSO "] = n30_blksplgapeg_dmso["Avg persistence length"]
pers_plgapegt_dmso

In [ ]:
rgplgapeg_olig_dmso.to_pickle("PLGAPEG_dmso_Rg.pkl")

In [ ]:
pers_plgapegt_dmso.to_pickle("PLGAPEG_dmso_Lp.pkl")

### Fluory Exponent, PLGA-PEG in DMSO systems 

In [ ]:
n_plgapeg = np.array([6,8,10,16,20,30])
rg_nplgapeg_dmso = np.array([np.mean(n6_blksplgapeg_dmso["Avg Radius of gyration"])
                    ,np.mean(n8_blksplgapeg_dmso["Avg Radius of gyration"]),np.mean(n10_blksplgapeg_dmso["Avg Radius of gyration"])
                    ,np.mean(n16_blksplgapeg_dmso["Avg Radius of gyration"]),
                    np.mean(n20_blksplgapeg_dmso["Avg Radius of gyration"]),np.mean(n30_blksplgapeg_dmso["Avg Radius of gyration"])])

In [ ]:
rg_nplgapeg_dmso

In [ ]:
rgdmso_nplgapeg_std = np.array([np.std(np.log10(n6_blksplgapeg_dmso["Avg Radius of gyration"]))
                    ,np.std(np.log10(n8_blksplgapeg_dmso["Avg Radius of gyration"]))
                        ,np.std(np.log10(n10_blksplgapeg_dmso["Avg Radius of gyration"]))
                       ,np.std(np.log10(n16_blksplgapeg_dmso["Avg Radius of gyration"]))     
                    ,np.std(np.log10(n20_blksplgapeg_dmso["Avg Radius of gyration"]))
                        ,np.std(np.log10(n30_blksplgapeg_dmso["Avg Radius of gyration"]))])
rgdmso_nplgapeg_std

In [ ]:
n_plgapeg

In [ ]:
np.log10(rg_nplgapeg_dmso)

In [ ]:
np.log10(n_plgapeg)

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_vdmso = LinearRegression(fit_intercept=True)
model_vdmso.fit(np.log10(n_plgapeg).reshape(-1,1), np.log10(rg_nplgapeg_dmso))
# Slope here is in nanometers
print("Model slope:    ", model_vdmso.coef_[0])
print("Model intercept:", model_vdmso.intercept_)

In [ ]:
gg_dmso = model_vdmso.predict(np.log10(n_plgapeg).reshape(-1,1))
gg_dmso

In [ ]:
print("Mean Std Error:", sklearn.metrics.mean_squared_error(np.log10(rg_nplgapeg_dmso), gg_dmso))
print("R2 score:", sklearn.metrics.r2_score(np.log10(rg_nplgapeg_dmso), gg_dmso))

In [ ]:
# Residuals between the true y data and model y data 
resid_vdmso = np.log10(rg_nplgapeg_dmso) - gg_dmso
resid_vdmso

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttdmso = np.log10(n_plgapeg)
nt_ttdmso -= np.mean(nt_ttdmso)
nhui_dmso = nt_ttdmso**2
np.sum(nhui_dmso)

In [ ]:
# t-value with 95 % confidence intervals 
scipy.stats.t.ppf(0.95, 4)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_vdmso = scipy.stats.t.ppf(0.95, 4)*np.sqrt((np.sum(resid_vdmso**2)/(len(resid_vdmso)-1))/(np.sum(nhui_dmso)))
flc_vdmso

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(n_plgapeg), np.log10(rg_nplgapeg_dmso), yerr=rgdmso_nplgapeg_std, color='#A58262', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(np.log10(n_plgapeg), gg_dmso, color='#A58262')
plt.title(r'Fluory Exponent', fontsize=15)
plt.xlabel(r'Log($N_{PLGA-PEG}$)', fontsize=15)
plt.ylabel(r'Log($R_{g}$)', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(1.1, 0.75, r'$v_{DMSO}$ = 0.61 ± 0.01', fontsize=15, color='#A58262')